In [1]:
# from google.colab import drive

# drive.mount('/content/drive')

In [2]:
#colab 환경에서 google drive에 업로드가 되지 않는 관계로, 저장 경로 PATH와 파일을 불러오는 경로 READ_PATH를 별도로 설정함. 실제 분석 상황에서는 PATH = READ_PATH로 통일하면 됨.
PATH = '../data/'
# READ_PATH = '/content/drive/MyDrive/final_data/'

In [ ]:
# !pip install py7zr
# !pip install newspaper3k
# !pip install yfinance


In [4]:
import torch
import py7zr
import numpy as np
import pandas as pd
from newspaper import Article

import os
import time
import re
import warnings

import yfinance as yf

warnings.filterwarnings("ignore")

# 1. RSS IFO 전처리 및 본문 크롤링
- 제공받은 nasdaq_rss_ifo의 데이터 중 본 분석에 맞도록 전처리 및 본문 크롤링을 실행함

## 행 필터링

In [ ]:
#7zip 파일
with py7zr.SevenZipFile((READ_PATH + 'NASDAQ_RSS_IFO.7z'), mode='r') as z:
     z.extractall(path=PATH)

In [ ]:
RSS_PATH = PATH + "NASDAQ_RSS_IFO"
os.listdir(RSS_PATH)

['NASDAQ_RSS_IFO_202305.csv',
 'NASDAQ_RSS_IFO_202307.csv',
 'NASDAQ_RSS_IFO_202303.csv',
 'NASDAQ_RSS_IFO_202306.csv',
 'NASDAQ_RSS_IFO_202304.csv',
 'NASDAQ_RSS_IFO_202308.csv',
 'NASDAQ_RSS_IFO_202302.csv',
 'NASDAQ_RSS_IFO_202301.csv']

In [ ]:
# 디렉토리가 없다면 생성
if not os.path.exists(RSS_PATH):
    os.makedirs(RSS_PATH)

# 모든 CSV 파일을 불러와서 하나의 데이터프레임으로 합치기
dfs = []
for filename in os.listdir(RSS_PATH):
    if filename.endswith('.csv'):
        month_df = pd.read_csv(os.path.join(RSS_PATH, filename), encoding='latin1')
        dfs.append(month_df)

# 모든 데이터프레임을 하나로 합치기
df = pd.concat(dfs, ignore_index=True)

# 중복 제거
df = df.drop_duplicates(keep='first')

In [ ]:
df.shape # (146914, 8)

(146914, 8)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146914 entries, 0 to 2294688
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   rgs_dt              146914 non-null  int64 
 1   tck_iem_cd          146914 non-null  object
 2   til_ifo             146914 non-null  object
 3   ctgy_cfc_ifo        146914 non-null  object
 4   mdi_ifo             146914 non-null  object
 5   news_smy_ifo        146914 non-null  object
 6   rld_ose_iem_tck_cd  146914 non-null  object
 7   url_ifo             146914 non-null  object
dtypes: int64(1), object(7)
memory usage: 10.1+ MB


In [ ]:
#url 기준 중복 행 제거
df['url_base'] = df['url_ifo'].apply(lambda x: re.sub(r'-0$', '', x))
print(df.shape)
df.drop_duplicates(subset=['url_ifo'], inplace=True)
print(df.shape)
df = df[~((df.duplicated(subset='url_base', keep=False)) & (df['url_ifo'].str.endswith('-0')))] #-0 앞까지 중복되면서, -0으로 끝나는 url이 들어간 행 모두 제거
print(df.shape)
df.drop(columns=['url_base'], inplace=True)
print(df.shape)

(146914, 9)
(93241, 9)
(89022, 9)
(89022, 8)


In [ ]:
#url이 "_"인 기사를 삭제함
#pre-market, after-hours 관련 기사는 단순히 주가를 나열하는 기사이므로 삭제함

df = df[~(df['url_ifo']=="_")]
print(df.shape)
df = df[~df['ctgy_cfc_ifo'].str.contains("Pre-Market|After-Hours")]
print(df.shape)
df = df[~df['til_ifo'].apply(lambda x: 'Pre-Market' in x)]
print(df.shape)
df = df[~df['til_ifo'].apply(lambda x: 'After-Hours' in x)]
print(df.shape)

(89021, 8)
(88769, 8)
(88676, 8)
(88602, 8)


In [ ]:
df.shape

(88602, 8)

### all_tck_iem_cd 열 생성

- 본래 tck_iem_cd 열의 주식을 대상으로 분석을 하려고 했으나, 해당 열이 정확하지 않다고 판단함.
- 예를 들어, 'AAPL'은 해당 열에 7, 8월에만 등장하는데, 이는 상식적이지 않음
- rld_ose_iem_tck_cd 열에는 'AAPL'이 지속적으로 등장하는 것으로 보아, 두 열의 티커코드를 합친 all_tck_iem_cd 열을 생성하여 이를 활용한 분석을 시행하는 게 적절하다고 판단함

In [ ]:
print('AAPL이 처음 등장하는 날짜: ', min(df[df['tck_iem_cd']=='AAPL']['rgs_dt']))
print('AAPL이 마지막으로 등장하는 날짜: ', max(df[df['tck_iem_cd']=='AAPL']['rgs_dt']))

AAPL이 처음 등장하는 날짜:  20230717
AAPL이 마지막으로 등장하는 날짜:  20230831


In [ ]:
# 'rld_ose_iem_tck_cd' 열에 있는 티커코드를 ','를 기준으로 분리하여 리스트로 만듦
df['rld_ose_iem_tck_cd_lst'] = df['rld_ose_iem_tck_cd'].str.split(',')

In [ ]:
print(min(df[df['rld_ose_iem_tck_cd_lst'].apply(lambda x: 'AAPL' in x)]['rgs_dt']))
print(max(df[df['rld_ose_iem_tck_cd_lst'].apply(lambda x: 'AAPL' in x)]['rgs_dt']))
print(len(df[df['rld_ose_iem_tck_cd_lst'].apply(lambda x: 'AAPL' in x)]['rgs_dt']))

20230111
20230831
937


In [ ]:
df[['rld_ose_iem_tck_cd', 'rld_ose_iem_tck_cd_lst']].head(10)

,rld_ose_iem_tck_cd,rld_ose_iem_tck_cd_lst
0,"CERT,CTMX","[CERT, CTMX]"
1,"TSLX,OCN","[TSLX, OCN]"
2,"PODD,DKNG,CERT,BITF","[PODD, DKNG, CERT, BITF]"
3,TMCI,[TMCI]
4,"NEO,WING","[NEO, WING]"
5,SLNA,[SLNA]
6,"OBT,SBNY","[OBT, SBNY]"
7,"NCLH,ISPO","[NCLH, ISPO]"
8,"NSA,DRH","[NSA, DRH]"
9,"AVDL,AVDL,RMTI,TGTX","[AVDL, AVDL, RMTI, TGTX]"


In [ ]:
#'tck_iem_cd'와 'rld_ose_iem_tck_cd_lst'에 있는 모든 주식코드를 all_tck_iem_cd 열에 합치고, 중복된 티커코드가 여러 번 나타나는 경우 제거함

df['all_tck_iem_cd'] = df.apply(lambda row: list(set([row['tck_iem_cd']] + row['rld_ose_iem_tck_cd_lst'])), axis=1)

In [ ]:
df[['tck_iem_cd', 'rld_ose_iem_tck_cd', 'all_tck_iem_cd']]

,tck_iem_cd,rld_ose_iem_tck_cd,all_tck_iem_cd
0,CERT,"CERT,CTMX","[CERT, CTMX]"
1,OCN,"TSLX,OCN","[OCN, TSLX]"
2,CERT,"PODD,DKNG,CERT,BITF","[CERT, PODD, DKNG, BITF]"
3,TMCI,TMCI,[TMCI]
4,NEO,"NEO,WING","[NEO, WING]"
...,...,...,...
2291846,LUNG,LUNG,[LUNG]
2291930,SMBK,SMBK,[SMBK]
2291973,PGC,PGC,[PGC]
2292219,HRTX,HRTX,[HRTX]


In [ ]:
df.drop('rld_ose_iem_tck_cd_lst', axis=1, inplace=True)

In [ ]:
df.columns

Index(['rgs_dt', 'tck_iem_cd', 'til_ifo', 'ctgy_cfc_ifo', 'mdi_ifo',
       'news_smy_ifo', 'rld_ose_iem_tck_cd', 'url_ifo', 'all_tck_iem_cd'],
      dtype='object')

In [ ]:
df.shape

(88602, 9)

In [ ]:
df.to_csv(os.path.join(PATH, 'nasdaq_df_wo_text.csv'), index=False)

In [ ]:
df = pd.read_csv(os.path.join(READ_PATH, "nasdaq_df_wo_text.csv"))
df.shape

(88602, 9)

## Text Crawling
- 제공받은 nasdaq_rss_ifo의 url을 newspaper 라이브러리를 이용해 기사본문을 크롤링함.

In [ ]:
!pip install newspaper3k

In [1]:
from newspaper import Article
import numpy as np
import pandas as pd
import time
import os
import re

In [ ]:
nasdaq_df = pd.read_csv(os.path.join(PATH, 'nasdaq_df_wo_text.csv'))
nasdaq_df.shape

(88602, 9)

In [2]:
def extract_text(url):

    article = Article(url)
    article.download()
    article.parse()
    text = article.text or 'N/A' #text가 없는 경우 'N/A'로 출력

    return text

In [4]:
test_url = 'https://biz.chosun.com/industry/company/2024/01/29/XFRL4GSDUJE65JJHYAIP4ILVZM/'
test_text = extract_text(test_url)
print(test_text)

N/A


In [ ]:
nasdaq_df['text'] = None  # 'text'열을 None으로 초기화

In [ ]:
nasdaq_df['text'].isna()

0        True
1        True
2        True
3        True
4        True
         ... 
88597    True
88598    True
88599    True
88600    True
88601    True
Name: text, Length: 88602, dtype: bool

In [ ]:
#크롤링 코드

from IPython.display import display

def crawl_nasdaq_texts(nasdaq_df, start_index, end_index): #start_index, end_index를 조정하여 여러 번에 나눠서 크롤링할 수 있음
    for index, row in nasdaq_df.iterrows():
        if index < start_index:
            continue

        try:
            url = row['url_ifo']
            extracted_text = extract_text(url)
            nasdaq_df.at[index, 'text'] = extracted_text

        #에러 나는 경우 처리(공백은 try 경우에 포함)
        except Exception as e:
            print(f"Error at index {index}: {e}")

        if index % 100 == 0:
            print(f"Processing index {index}")
            display(nasdaq_df.iloc[index-10:index])

        #아래 코드는 ckpt를 사용할 경우 주석 제거하시면 됩니다.

        # if index % 1000 == 0:
        #     folder_path = f'{PATH}'"/nasdaq_text_crawling_ckpt"
        #     if not os.path.exists(folder_path):
        #         os.makedirs(folder_path)
        #     nasdaq_df.to_csv(os.path.join(PATH, "nasdaq_text_crawling_ckpt", f"{start_index}_{index}_ckpt.csv"), index=False)

        #index > end_index인 경우 자동으로 제거
        if index == end_index:
            folder_path = f'{PATH}'"/nasdaq_text_crawling_ckpt"
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            nasdaq_df.to_csv(os.path.join(PATH, "nasdaq_text_crawling_ckpt", f"{start_index}_{index}_ckpt.csv"), index=False)
            print(f"{start_index}_{index} 크롤링 완료")
            break

    return nasdaq_df

In [ ]:
result_df = crawl_nasdaq_texts(nasdaq_df, 0, len(nasdaq_df)-1)

In [ ]:
# #ckpt를 사용하고, 범위를 나눠서 크롤링할 경우의 코드.

# #ckpt_path에 있는 모든 파일을 하나의 데이터프레임으로 합치기

# CKPT_PATH = os.path.join(PATH, "nasdaq_text_crawling_ckpt")

# csv_files = [f for f in os.listdir(CKPT_PATH) if f.endswith('.csv')]

# min_val, max_val = float('inf'), float('-inf')

# # 각 CSV 파일을 읽어서 처리
# for i, csv_file in enumerate(csv_files):
#     # 데이터프레임 로드
#     df = pd.read_csv(os.path.join(CKPT_PATH, csv_file))

#     if i == 0:
#         result_df = df.copy()
#         result_df[f'text_{i}'] = df['text']

#     else:
#         result_df[f'text_{i}'] = df['text'] #새로운 열에 text값 추가

#     numbers = re.findall(r'\d+', csv_file)  # 숫자를 모두 찾음
#     min_candidate, max_candidate = map(int, numbers[:2])  #(0과 20000, 50000과 73000 등 파일명의 숫자를 찾음)

#     # 최소값과 최대값 업데이트
#     min_val = min(min_val, min_candidate)
#     max_val = max(max_val, max_candidate)

# print(f"Results ranging from {min_val} to {max_val}")
# print()
# result_df.head()

In [ ]:
#중복 행 제거
result_df.drop_duplicates(inplace=True)
result_df.shape

In [ ]:
# 'text'로 시작하는 모든 열을 필터링
text_columns = [col for col in result_df.columns if col.startswith('text')]
print(text_columns)

# 해당 열들에서 'NaN'(에러가 난 행)이 아닌 첫 번째 값을 찾는 새로운 열 생성. 모두 'NaN'일 경우, 'NaN'으로 저장
result_df['text_not_nan'] = result_df[text_columns].apply(lambda row: next((item for item in row if not pd.isna(item)), np.nan), axis=1)

result_df.head()

In [ ]:
result_df = result_df[result_df['text_not_nan']!='N/A'] #text_not_nan이 'N/A'가 아닌 행만 필터링(text가 공백인 행)

In [ ]:
result_df.isna().sum() #text_not_nan에 null값(error난 행) 있는지 확인.

In [ ]:
result_df.drop(text_columns, axis=1, inplace=True) #바로 위에서 정의한 text로 시작하는 열 drop(text_not_nan은 drop 안 됨)

result_df.rename(columns={'text_not_nan': 'text'}, inplace=True)

result_df.head()

In [ ]:
#error난 url 다시 크롤링 시도한 후 최종 결과를 nasdaq_final.csv에 저장

nan_index_range = result_df[result_df['text'].isna()].index
print("NaN indices: ", nan_index_range)

def recrawl_nasdaq_error_texts(nasdaq_df, start_index=0, end_index=len(df)-1, nan_index_range=None): #원하는 index range에서 nan_index_range 찾을 수 있음
    error_url_list = []
    for index in nan_index_range:
        if index < start_index:
            continue

        # #end_index를 작게 설정하는 경우 대비. end_index=len(df)-1이라면 필요 없음.
        # if index > end_index:
        #     nasdaq_df.to_csv(os.path.join(PATH, "nasdaq_final.csv"), index=False)
        #     print(f"{start_index}_{end_index} 크롤링 완료")
        #     break


        try:
            url = nasdaq_df.at[index, 'url_ifo']
            extracted_text = extract_text(url)
            nasdaq_df.at[index, 'text'] = extracted_text
            print(f"Processing index {index}")

        except Exception as e:
            print(f"Error at index {index}: {e}")
            error_url_list.append(url)

    #error가 난 url이 있는 행 포함하여 저장
    print("Number of error urls: ", len(error_url_list))
    nasdaq_df.to_csv(os.path.join(PATH, "nasdaq_final.csv"), index=False)

    return nasdaq_df, error_url_list

In [ ]:
df, error_url_list = recrawl_nasdaq_error_texts(result_df, nan_index_range=nan_index_range)

# 2. Stock Description Crawling
- 야후파이낸스의 라이브러리를 이용해 기업 설명이 기재된 description을 크롤링함.
- 추후 토픽의 키워드와 키워드에 맞는 기업을 연결하기 위함.

In [8]:
import yfinance as yf
import pandas as pd
import warnings
import os
warnings.filterwarnings("ignore")

In [ ]:
#PATH #경로가 올바르게 설정되어 있는지 확인

'/content/'

In [ ]:
stock = pd.read_csv(os.path.join(READ_PATH, 'NASDAQ_FC_STK_IEM_IFO.csv'), encoding = "cp949")
stock

,isin_cd,tck_iem_cd,fc_sec_krl_nm,fc_sec_eng_nm
0,US00211V1061,AACG,ATA ...,ATA CreatGlo ...
1,US00032Q1040,AADI,Aadi Bioscience ...,Aadi Bioscience ...
2,US02376R1023,AAL,아메리칸 에어라인스 그룹 ...,American Airline ...
3,US03823U1025,AAOI,어플라이드 옵토일렉트로닉스 ...,AOI ...
4,US0003602069,AAON,에이에이온 ...,AAON ...
...,...,...,...,...
2738,US4884452065,ZVRA,Zevra ...,Zevra ...
2739,US98987D1028,ZVSA,Zyversa ...,Zyversa ...
2740,US98985Y1082,ZYME,Zymeworks ...,Zymeworks ...
2741,US98986X1090,ZYNE,자이너바 파마수티컬스 ...,Zynerba Pharms ...


In [ ]:
# 'tck_iem_cd' 열의 공백 제거
stock['tck_iem_cd'] = stock['tck_iem_cd'].str.strip()

In [ ]:
# 종목 티커 코드 리스트
tck_iem_cds = list(stock['tck_iem_cd'])

In [ ]:
# 종목 description 추출
def get_stock_descriptions(tck_iem_cds):
    descriptions = []

    for tck_iem_cd in tck_iem_cds:
        try:
            # Ticker 객체 생성
            ticker = yf.Ticker(tck_iem_cd)

            # 종목 정보 가져오기
            stock_info = ticker.info

            # 'longBusinessSummary' 키에 해당하는 종목 description 정보 가져오기
            description = stock_info.get('longBusinessSummary', None)

            # 가져온 정보를 리스트에 추가
            descriptions.append(description)
        except Exception as e:
            print(f"Error fetching data for {tck_iem_cd}: {str(e)}")
            descriptions.append(None)

    data = pd.DataFrame({'tck_iem_cd': tck_iem_cds, 'description': descriptions})

    return data

In [ ]:
#데이터프레임 생성
stock_description = get_stock_descriptions(tck_iem_cds)

stock_description

In [ ]:
#결측치 확인
stock_description.isnull().sum()

In [ ]:
#전처리
stock_description = stock_description.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
stock_description.dropna(subset=['description'], inplace=True)
stock_description.reset_index(drop=True, inplace=True)
stock_description.drop_duplicates(subset=['description'], inplace=True)

In [ ]:
stock_description.head()

In [ ]:
stock_description.shape

### 데이터프레임에 영문기업명 추가
- 추후 FinBERT로 감성분석 진행시 영문기업명을 기준으로 기사 문단을 추출하기 위함.

In [ ]:
company_name = stock[['tck_iem_cd', 'fc_sec_eng_nm']]
company_name

In [ ]:
# 두 데이터프레임을 tck_iem_cd 열을 기준으로 inner join
stock_description_new = pd.merge(stock_description, company_name, how='inner', on='tck_iem_cd')
stock_description_new.head()

In [ ]:
stock_description_new.shape

In [ ]:
stock_description_new.to_csv(os.path.join(PATH, 'stock_description.csv'), index=False)

# 3. CNBC 기사 url 크롤링
- 크롤링 목적 : 토픽모델링 시에 사용할 데이터
- 데이터 선정 이유: CNBC는 미국의 경제·금융 뉴스 채널로, CNBC에서 각광받거나 언급량이 많은 토픽은 시장에서 많은 관심을 받고 있는 분야라고 여길 수 있다. 그렇기에 투자 기업을 정하기 전 CNBC를 통해 앞으로 성장 가능성이 있거나 시장의 개입이 이뤄질 것 같은 분야를 먼저 확인하고, 해당 분야에서 투자 가치가 있는 기업을 제공하고자 한다.

- 수집 데이터 일자 : 2023.1.1 ~ 2023.8.31

- 크롤링 방식 : Google에 "CNBC"를 검색하고, 7일 단위(2023.1.1 ~ 2023.1.7, ...)로 필터링을 주어, 해당 주의 모든 기사 url 크롤링

- 7일 단위로 필터링한 이유 : 구글 검색 결과의 제한(33페이지)으로 인해 기간을 길게 할 경우, 해당 기간의 기사가 모두 표시되지 않기 때문, 기간을 너무 짧게 할 경우에는 필요없는 기사가 추출됨을 확인했기에 7일이 적절하다고 판단.

- driver가 colab 환경에서 실행되지 않아, <CNBC 기사 url 크롤링> 섹션에서는 가상환경에서 실행하셔야 합니다.
(가능하다면 colab 환경에서 실행하셔도 됩니다.)
- conda create -n NH python=3.8
- selenium==4.12.0, pandas==2.0.3, re==2.2.1, bs4==4.12.2, requests==2.31.0, newspaper==0.2.8

In [ ]:
!pip install selenium==4.12.0
!pip install bs4
!pip install requests
!pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=52d77468fd6d345fe82c0fd083bf3902925ac79abd7eedf371149b6c952ff761
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from datetime import datetime, timedelta
import pickle
import os
from bs4 import BeautifulSoup
import requests
from newspaper import Article

In [ ]:
#해당 경로는 colab의 READ_PATH, PATH의 역할을 합니다

PATH = "./"

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)


def cnbc_crawling(start_date, end_date):

    global driver

    start_values_iter = iter(range(0, 320, 10)) #페이지 number - 0이면 10페이지
    current_matching_links = []
    retry_count = 0
    terminate_flag = False

    while True:
        if terminate_flag:
            break

        #구글 검색결과의 각 페이지에서 크롤링
        try:
            current_start = next(start_values_iter)
        except StopIteration:
            break

        print(f"page: {current_start // 10 + 1}")

        while True:
            matching_links_count = 0
            date_range_str = f"cd_min:{start_date},cd_max:{end_date}"
            google_search_url = f'https://www.google.com/search?q=cnbc&sca_esv=568517199&tbs=cdr:1,{date_range_str}&tbm=nws&sxsrf=AM9HkKmYdZ0_zRMPireMpaE6VsIgqW5jBg:1695740222154&ei=PvESZZmECbfh2roPnbOa-A0&start={current_start}&sa=N&ved=2ahUKEwiZm8rMxMiBAxW3sFYBHZ2ZBt84ygIQ8tMDegQIAxAW&biw=1137&bih=790&dpr=2'
            driver.get(google_search_url)

            links_with_class = driver.find_elements(By.CSS_SELECTOR, 'a.WlydOe[jsname="YKoRaf"]')
            for link in links_with_class:
                href = link.get_attribute('href')

                if href and re.match(r'https://www\.cnbc\.com/2023/\d{2}/\d{2}/[a-z0-9-]+\.html', href):
                    current_matching_links.append(href)
                    print(href)
                    matching_links_count += 1

            if matching_links_count > 0:
                retry_count = 0
                break

            #해당 페이지에 matching되는 url이 없으면 재시도 - "로봇이 아닙니다"가 뜨거나 페이지가 비었다면, matching되는 url이 없을 것
            else:
                driver.quit()
                driver = webdriver.Chrome(options=chrome_options)
                retry_count += 1

            #재시도 횟수가 2번 이상이면 terminate시키고 다음 기간으로 넘어감. (그 페이지가 비었다는 뜻이므로). 재시도란, 결과가 나오지 않아 드라이버를 껐다 켜는 행위.
            #"로봇이 아닙니다"는 driver를 새로 열면 사라지므로, 재시도 횟수가 2번 이상이라는 것은 정말 페이지가 비었다는 의미.
            if retry_count >= 2:
                terminate_flag = True
                break

    return current_matching_links

#함수 호출 예시 - 7일 단위로 날짜 설정해주기
#cnbc_crawling("01/01/2023", "01/07/2023")

SessionNotCreatedException: ignored

In [ ]:
#7일 단위로 크롤링 - 체크포인트 코드 제거

start_date = datetime.strptime("1/1/2023", "%m/%d/%Y")
end_date = datetime.strptime("8/31/2023", "%m/%d/%Y")
all_matching_links = [] #모든 link 저장
empty_count = 0

# Driver options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)

# 7일 범위 날짜 생성
while start_date <= end_date:
    next_end_date = start_date + timedelta(days=6)
    if next_end_date > end_date:
        next_end_date = end_date

    #7일 단위로 크롤링
    current_start_date, current_end_date = (start_date.strftime("%m/%d/%Y"), next_end_date.strftime("%m/%d/%Y"))
    current_matching_links = cnbc_crawling(current_start_date, current_end_date)
    all_matching_links += current_matching_links

    # 결과 처리 또는 저장
    print(f'크롤링 결과 (시작 날짜: {current_start_date}, 종료 날짜: {current_end_date}):')
    print(f'누적 크롤링된 기사 수: {len(all_matching_links)}')
    print(f'현재 크롤링된 기사 수: {len(current_matching_links)}')

    start_date += timedelta(days=7)


    # ###4주마다 체크포인트 저장
    # week_counter += 1  # Increment week_counter
    # if week_counter == 4:  # Check if 4 weeks have passed
    #     file_name = f"all_matching_links_{next_end_date.strftime('%Y_%m_%d')}.pkl"
    #     full_path = os.path.join(PATH, file_name)
    #     with open(full_path, 'wb') as f:
    #         pickle.dump(all_matching_links, f)
    #     week_counter = 0  # Reset week_counter

    # Save the remaining data if next_end_date reaches end_date

if next_end_date == end_date:
    file_name = "all_matching_links_final.pkl"
    full_path = os.path.join(PATH, file_name)
    with open(full_path, 'wb') as f:
            pickle.dump(all_matching_links, f)

## url에 접속하여 기사 정보 크롤링하기

- url에 접속하여, 제목, 날짜, 카테고리, Key Points(기사요약), 본문 크롤링
- 카테고리 크롤링 이유 : 분석 시, 주식 정보에 관련 없는 카테고리는 삭제하기 위함.
- Key Points 크롤링 이유 : 기사 요약본으로 토픽모델링을 진행하고자 했으나, 문자열이 짧아 토픽이 잘 추출되지 않았음.
- 본문 크롤링 이유 : 토픽모델링에 실제로 사용한 데이터, 본문 데이터로 토픽모델링을 진행한 결과 토픽이 가장 잘 추출되었음.

In [ ]:
all_matching_links = pd.read_pickle(os.path.join(READ_PATH, "all_matching_links_final.pkl"))

CNBC_CKPT_PATH = os.path.join(PATH, "news_info_ckpt")
if not os.path.exists(CNBC_CKPT_PATH):
    os.makedirs(CNBC_CKPT_PATH, exist_ok=True)


#Key Points 크롤링
def extract_key_points(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        target_div = soup.find('div', {'class': 'group'})
        if target_div:
            ul = target_div.find('ul')
            if ul:
                lis = ul.find_all('li')
                return ' '.join([li.text for li in lis])
    except Exception as e:
        print(f"Error in extract_key_points: {e}")
        return ""


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)

# 기사의 정보를 저장할 list 생성
articles_data = []


def extract_article_info(url, retry_count=0): #재시도 횟수 0에서 시작
    global driver  # 드라이버를 글로벌 변수로 설정

    #category 크롤링
    try:
        driver.get(url)
        page_html = driver.page_source
        soup = BeautifulSoup(page_html, 'html.parser')

        article_header = soup.find(class_='ArticleHeader-eyebrow') or soup.find(class_='ArticleHeader-styles-makeit-eyebrow--Degp4')

        category = article_header.text if article_header else "N/A"
    except Exception as e:
        print(f"Error in category extraction: {e}")
        category = ""


    #title, date, key_points, text 크롤링
    try:
        article = Article(url)
        article.download()
        article.parse()
        title = article.title or 'N/A'
        date = article.publish_date or 'N/A'
        text = article.text or 'N/A'
        key_points = extract_key_points(url) or 'N/A'


    except Exception as e:
        print(f"Error in newspaper extraction: {e}")

        # 에러가 발생하면 드라이버 재시작
        driver.quit()
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')
        driver = webdriver.Chrome(options=chrome_options)

        # retry_count < 1이면 크롤링 다시 시도
        if retry_count < 1:
            return extract_article_info(url, retry_count=retry_count + 1) #재귀적 코드

        #retry_count >= 1일 때, url을 제외한 행 전체가 빈칸으로 나타남

        print(f'Error in {url}')
        title, date, text, key_points = "", "", "", ""

    new_data = {
        'title': title,
        'date': date,
        'category': category,
        'key_points': key_points,
        'text': text,
        'url': url
    }

    #새로 크롤링한 데이터를 articles_data list에 추가
    articles_data.append(new_data)
    #print(f"Newly appended data: {new_data}")

#url list 정의 - link를 날짜 순서대로 정렬
url_list = sorted(list(all_matching_links))

for idx, url in enumerate(url_list, 1):
    extract_article_info(url)

    #10개 단위로 체크포인트 파일 저장
    if idx % 10 == 0:
        checkpoint_df = pd.DataFrame(articles_data)
        display(checkpoint_df)

        with open(os.path.join(CNBC_CKPT_PATH, f"{idx}번째_체크포인트.pkl"), 'wb') as f:
            pickle.dump(checkpoint_df, f)
        print(f"Saved checkpoint at {idx}th URL.")

    #time.sleep(1.5)

driver.quit()

df = pd.DataFrame(articles_data)

df

In [ ]:
df[(df=='').any(axis=1)]

In [ ]:
df_filtered = df[df['title'] != '']

df_filtered

In [ ]:
with open(os.path.join(PATH, "cnbc_newsdata_final.pkl"), 'wb') as f:
    pickle.dump(df_filtered, f)

## 크롤링 완료 후 데이터 저장 링크
https://drive.google.com/drive/folders/1I-pmi33w6z_hUanRpHdbuSRRXG7LN79v?usp=sharing

# 4. BERTopic을 이용한 8월 CNBC기사 토픽모델링
- CNBC 경제 뉴스 사이트에서 최근 인기 있는 기사 토픽을 파악하고자 함.
- 대회 시작 당시 9월 초였기에 '최근'의 기준을 8월 1달로 한정함.

In [89]:
READ_PATH = "/content/drive/MyDrive/final_data/"
PATH = "/content/"

In [121]:
# !pip install spacy
# !pip install matplotlib
# !pip install seaborn
# !pip install tqdm
# !pip install sentence_transformers
# !pip install hdbscan
!pip install bertopic
# !pip install scikit-learn
!pip install plotly

In [91]:
#!pip install umap-learn

In [124]:
# #전처리 패키지
import pandas as pd
import numpy as np
from datetime import datetime
import re
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# #토픽 모델링 패키지
from sentence_transformers import SentenceTransformer
# from umap.umap_ import UMAP
from bertopic import BERTopic
import hdbscan
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from bertopic.representation import MaximalMarginalRelevance

#interactive plot 시각화
import plotly.express as px

import plotly.io as pio
pio.renderers.default = "notebook_connected"

## 데이터 준비

In [93]:
PATH

'/content/'

In [94]:
import os

In [95]:
df = pd.read_pickle(os.path.join(READ_PATH, 'cnbc_newsdata_final.pkl'))
df

,title,date,category,key_points,text,url
0,‘I work just 5 hours a week': A 39-year-old wh...,2023-01-01,Success,N/A,"Graham Cochrane, Founder of The Recording Revo...",https://www.cnbc.com/2023/01/01/39-year-old-wh...
1,Chinese state media seek to reassure public ov...,2023-01-01,Asia-Pacific News,Chinese state media sought to reassure the pub...,Revelers prepare to release balloons to celebr...,https://www.cnbc.com/2023/01/01/chinese-state-...
2,Should you get creative with your resume? Expe...,2023-01-01,Land the Job,N/A,Mature businessman congratulating young profes...,https://www.cnbc.com/2023/01/01/cv-will-a-crea...
3,Market misery deals sovereign wealth funds his...,2023-01-01,Markets,Heavy falls in stock and bond markets over the...,A trader works on the floor of the New York St...,https://www.cnbc.com/2023/01/01/market-misery-...
4,More social media regulation is coming in 2023...,2023-01-01,Tech,Days after Congress passed a bipartisan spendi...,"The U.K.'s Online Safety Bill, which aims to r...",https://www.cnbc.com/2023/01/01/more-social-me...
...,...,...,...,...,...,...
5628,63% of workers unable to pay a $500 emergency ...,2023-08-31,Personal Finance,Workers are reporting financial stress amid hi...,A shopper makes their way through a grocery st...,https://www.cnbc.com/2023/08/31/63percent-of-w...
5629,"This 22-year-old earns $194,000 at Google and ...",2023-08-31,Millennial Money,N/A,This story is part of CNBC Make It's Millennia...,https://www.cnbc.com/2023/08/31/22-year-old-ea...
5630,China's factory activity shrinks for a fifth s...,2023-08-31,China Economy,The official manufacturing purchasing managers...,People walk through a gate in the Forbidden Ci...,https://www.cnbc.com/2023/08/31/china-economy-...
5631,Southeast Asia's first high-speed train – a bo...,2023-08-31,Access ASEAN,A 142-kilometre rail line linking Jakarta with...,Indonesia is starting trial runs for its first...,https://www.cnbc.com/2023/08/31/indonesias-chi...


In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5633 entries, 0 to 5632
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   title       5633 non-null   object        
 1   date        5633 non-null   datetime64[ns]
 2   category    5633 non-null   object        
 3   key_points  5633 non-null   object        
 4   text        5633 non-null   object        
 5   url         5633 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 264.2+ KB


In [97]:
#잘못 크롤링된 9월 데이터 제거
df = df[df['date'].dt.strftime('%Y-%m') != '2023-09']
df.shape

(5632, 6)

## 카테고리 값 중 월별로 5회 미만 언급된 기사 drop
- 월에 5번도 언급되지 않은 카테고리는 주가 분석에 있어 중요하지 않은 기사라고 판단해 삭제함.

In [98]:
# 'date' 컬럼에서 월 정보를 추출하여 'month' 컬럼 생성
df['month'] = df['date'].dt.month

# 월 별로 카테고리 등장 횟수 계산
monthly_counts = df.groupby(['month', 'category']).size().reset_index(name='count')

# 월 별로 5번 이상 등장한 카테고리 찾기
valid_categories = monthly_counts[monthly_counts['count'] >= 5]['category'].unique()

final_df = df[df['category'].isin(valid_categories)]

# 결과 출력
final_df

,title,date,category,key_points,text,url,month
0,‘I work just 5 hours a week': A 39-year-old wh...,2023-01-01,Success,N/A,"Graham Cochrane, Founder of The Recording Revo...",https://www.cnbc.com/2023/01/01/39-year-old-wh...,1
1,Chinese state media seek to reassure public ov...,2023-01-01,Asia-Pacific News,Chinese state media sought to reassure the pub...,Revelers prepare to release balloons to celebr...,https://www.cnbc.com/2023/01/01/chinese-state-...,1
2,Should you get creative with your resume? Expe...,2023-01-01,Land the Job,N/A,Mature businessman congratulating young profes...,https://www.cnbc.com/2023/01/01/cv-will-a-crea...,1
3,Market misery deals sovereign wealth funds his...,2023-01-01,Markets,Heavy falls in stock and bond markets over the...,A trader works on the floor of the New York St...,https://www.cnbc.com/2023/01/01/market-misery-...,1
4,More social media regulation is coming in 2023...,2023-01-01,Tech,Days after Congress passed a bipartisan spendi...,"The U.K.'s Online Safety Bill, which aims to r...",https://www.cnbc.com/2023/01/01/more-social-me...,1
...,...,...,...,...,...,...,...
5625,Biden says he will visit Florida this weekend ...,2023-08-31,Politics,President Joe Biden said he would be traveling...,WASHINGTON — President Joe Biden made a surpri...,https://www.cnbc.com/2023/08/31/biden-says-he-...,8
5626,Baidu's Ernie bot jumps to the top of Apple's ...,2023-08-31,Tech,Chinese tech giant Baidu announced Thursday it...,Pictured here is the Ernie bot mobile interfac...,https://www.cnbc.com/2023/08/31/baidu-gets-chi...,8
5627,Apple reportedly tests 3D printing to manufact...,2023-08-31,Tech,Apple is reportedly testing using 3D printing ...,Apple is testing the use of 3D printers to mak...,https://www.cnbc.com/2023/08/31/apple-is-testi...,8
5628,63% of workers unable to pay a $500 emergency ...,2023-08-31,Personal Finance,Workers are reporting financial stress amid hi...,A shopper makes their way through a grocery st...,https://www.cnbc.com/2023/08/31/63percent-of-w...,8


In [99]:
# 2023년 8월 이후 데이터만 선택
start_date = '2023-08-01'
final_df = final_df[final_df['date'] >= start_date]
final_df.shape

(616, 7)

In [100]:
final_df.date

4946   2023-08-01
4947   2023-08-01
4948   2023-08-01
4949   2023-08-01
4950   2023-08-01
          ...    
5625   2023-08-31
5626   2023-08-31
5627   2023-08-31
5628   2023-08-31
5630   2023-08-31
Name: date, Length: 616, dtype: datetime64[ns]

## 데이터 전처리
- 크롤링 데이터의 특수문자와 문장부호 제거
- 불용어 제거(n차 토픽모델링 후 후처리 진행)
- 불용어 제거 및 토큰화 : 상업용으로 사용하기에 사용성과 성능 면에서 nltk보다 spacy가 우수해 spacy 모델을 사용했다. 긴 텍스트(기사 본문)를 처리해야 하므로 en_core_web_lg 모델을 사용했다. 또한 tokenizer 함수를 생성할 때는 명사만 추출하도록 했으며, 개체명은 'TIME','CARDINAL','DATE'을 제외한 모든 entity를 사용했다.

In [101]:
# 특수 문자와 문장 부호 제거 함수 정의
def remove_special_characters(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    return cleaned_text

# 데이터 전처리 함수 적용
final_df['text'] = final_df['text'].apply(remove_special_characters)

In [102]:
#spacy 모델 다운로드
!python -m spacy download en_core_web_lg

2023-10-15 21:54:51.886837: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [103]:
#spacy 모델 정의
nlp = spacy.load("en_core_web_lg")

#불용어 리스트
spacy_stopwords_list = list(nlp.Defaults.stop_words)

#처리할 문자열 최대 길이
nlp.max_length = 10000000

#토픽모델링 후 추가 불용어 삭제
new_stopwords_list = ['cnbc', 'share', 'earning', 'revenue', 'premarket', 'stock', 'chart icon', 'stock chart', 'chart', 'icon', 'company', 'store', 'wedding', 'food', 'economy', 'bank']
spacy_stopwords_list_new = spacy_stopwords_list + new_stopwords_list

In [104]:
#tokenizer 함수 생성
#1. 명사 추출
#2. 소문자 변환
#3. 불용어 제거
#4. 개체명 time, cardinal, date 제거
def tokenizer(text):
    words=[]
    doc=nlp(text)
    for token in doc:
        if token.tag_[0] in ['N'] and token.lemma_.lower() not in spacy_stopwords_list_new and token.ent_type_ not in ['TIME','CARDINAL','DATE']:
                if len(token.lemma_.lower())>1:
                    words.append(token.lemma_.lower())

    return words

## 토픽모델링
- 임베딩 모델 선정 기준 : sbert.net의 sentencetransformer 중 가장 Performance가 높은 모델(all-mpnet-base-v2) 선정
- 5배나 빠른 속도에 정확도가 높은 all-MiniLM-L6-v2로도 시도해봤지만 성능이 좋지 않았음.

In [105]:
#topic_modeling data 생성
all_texts = final_df['text'].values #전체 뉴스 데이터
timestamps = final_df['date'].to_list() #뉴스 발행일자

In [106]:
embedding_model = SentenceTransformer("all-mpnet-base-v2") #임베딩 모델
embeddings = embedding_model.encode(all_texts, show_progress_bar=True) #임베딩 미리 계산(파라미터 수정 용이 위함)

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

**파라미터 튜닝 결과 아래의 파라미터로 하는 것이 가장 토픽을 잘 찾는다고 판단함.**
- UMAP(n_neighbors=8, min_dist=0.1, n_components=2)
- HDBSCAN(min_cluster_size=5)


- TfidfVectorizer 사용 이유 : 단어의 빈도 뿐만 아니라, 그 단어가 전체 문서 집합에서 얼마나 중요한지를 고려하기에 토픽모델링 시에 해당 모델을 사용하는 것이 적합함.(문서 집합에서 단어의 출현 빈도만을 고려하는 CountVectorizer는 불용어까지 토픽 키워드로 선정할 수 있다고 판단.)
- MaximalMarginalRelevance 사용 이유 : 토픽의 키워드를 통해 관련주를 찾아내야 하므로, 토픽 키워드를 추출하는 것이 정교해야 한다고 판단함. 또한 diversity를 0.2로 설정해 토픽과 관련된 키워드를 다소 다양하게 뽑고자 했음.(여러 번의 시행착오 결과 최적의 파라미터가 0.2라고 판단.)

In [108]:
from umap.umap_ import UMAP

In [109]:
def pipeline_models():
    # UMAP 모델 정의
    umap_model = UMAP(n_neighbors=8, min_dist=0.1, n_components=2, random_state=42, metric='cosine')

    # HDBSCAN 모델 정의
    hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

    # TfidfVectorizer 모델 정의
    vectorizer_model = TfidfVectorizer(tokenizer=tokenizer, stop_words="english", ngram_range=(1, 2), min_df=2)

    # MaximalMarginalRelevance 모델 정의
    representation_model = MaximalMarginalRelevance(diversity=0.2)

    return umap_model, hdbscan_model, vectorizer_model, representation_model

# 모델 생성
umap_model, hdbscan_model, vectorizer_model, representation_model = pipeline_models()

In [110]:
def create_topic_model(embedding_model, umap_model, hdbscan_model, vectorizer_model, representation_model):
    # BERTopic 모델 정의
    topic_model = BERTopic(
        # Pipeline models
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        representation_model=representation_model,
        # Hyperparameters
        top_n_words=10,
        min_topic_size=5,
        verbose=True
    )

    return topic_model

# 모델 생성
topic_model = create_topic_model(embedding_model, umap_model, hdbscan_model, vectorizer_model, representation_model)

In [111]:
topics, probs = topic_model.fit_transform(all_texts, embeddings)

2023-10-15 22:16:06,996 - BERTopic - Reduced dimensionality
2023-10-15 22:16:07,047 - BERTopic - Clustered reduced embeddings


In [112]:
#share값과 토픽과 연관된 문장의 단어
def get_topic_stats(topic_model):
    topics_info_df = topic_model.get_topic_info().sort_values('Count', ascending = False)
    topics_info_df['Share'] = 100.*topics_info_df['Count']/topics_info_df['Count'].sum()
    topics_info_df['CumulativeShare'] = 100.*topics_info_df['Count'].cumsum()/topics_info_df['Count'].sum()
    return topics_info_df[['Topic', 'Count', 'Share', 'CumulativeShare', 'Name', 'Representation']]

In [113]:
#상위 20개 토픽 추출
topic_stat = get_topic_stats(topic_model).head(20).set_index('Topic')
topic_stat

,Count,Share,CumulativeShare,Name,Representation
Topic,,,,,
-1,128,20.779221,20.779221,-1_bitcoin_resume_oracle_market,"[bitcoin, resume, oracle, market, sec, etf, ap..."
0,34,5.519481,26.298701,0_analyst refinitiv_analyst_cramer_price target,"[analyst refinitiv, analyst, cramer, price tar..."
1,29,4.707792,31.006494,1_ai_chatgpt_google_ai model,"[ai, chatgpt, google, ai model, openai, chatbo..."
2,28,4.545455,35.551948,2_retailer_walmart_foot locker_merchandise,"[retailer, walmart, foot locker, merchandise, ..."
3,24,3.896104,39.448052,3_pfizer_vaccine_pharmacy_medication,"[pfizer, vaccine, pharmacy, medication, obesit..."
4,24,3.896104,43.344156,4_cnn_disney_microsoft_activision,"[cnn, disney, microsoft, activision, espn, sal..."
5,23,3.733766,47.077922,5_election_president donald_indictment_case,"[election, president donald, indictment, case,..."
6,21,3.409091,50.487013,6_iphone_apple_ipad_smartphone,"[iphone, apple, ipad, smartphone, apple iphone..."
7,20,3.246753,53.733766,7_playlist_schwartz_taylor_feedback,"[playlist, schwartz, taylor, feedback, billion..."


### 기술 관련 토픽의 키워드 확인
- 토픽을 확인해본 결과, 토픽 0, 3, 6, 11, 15이 기술과 관련이 있다고 판단함

In [114]:
topic_model.get_topic(0)

[('analyst refinitiv', 0.03372675178059123),
 ('analyst', 0.03358069195842215),
 ('cramer', 0.029788355301943913),
 ('price target', 0.02431798295350915),
 ('cent analyst', 0.02416615560766607),
 ('analyst factset', 0.01646362837866513),
 ('wall street', 0.01471902800971651),
 ('nvidia', 0.0134541032796249),
 ('refinitiv cent', 0.01326958911577641),
 ('trade alert', 0.013018680675714253)]

In [115]:
topic_model.get_topic(3)

[('pfizer', 0.03984946568083507),
 ('vaccine', 0.036666046213431495),
 ('pharmacy', 0.031354301864470226),
 ('medication', 0.031088154769561154),
 ('obesity', 0.02947804525251283),
 ('novo', 0.023910544694010386),
 ('cvs', 0.02173974013150382),
 ('telegram', 0.020354560583392255),
 ('marketplace', 0.020174584958086234),
 ('covid', 0.017711835197507627)]

In [116]:
topic_model.get_topic(6)

[('iphone', 0.12257195997513622),
 ('apple', 0.106950452167445),
 ('ipad', 0.04105567881255938),
 ('smartphone', 0.040866859685472635),
 ('apple iphone', 0.035399444958679),
 ('huawei', 0.03232852302698514),
 ('device', 0.03137481122840396),
 ('tablet', 0.02255925250547163),
 ('android', 0.02098821779092229),
 ('iphone model', 0.019903119080530076)]

In [117]:
topic_model.get_topic(11)

[('oil', 0.0765774770724255),
 ('vessel', 0.07464874138775855),
 ('port', 0.02821272545509403),
 ('sailing', 0.02493859355202305),
 ('coast', 0.02378349325924912),
 ('gulf', 0.021537729715534585),
 ('ocean', 0.01934077079955524),
 ('el niño', 0.019109865744352927),
 ('waterway', 0.018559762799953514),
 ('supply', 0.01829513187074302)]

In [118]:
topic_model.get_topic(15)

[('rent', 0.06075332393046261),
 ('city', 0.048954732448530394),
 ('new york', 0.034744671447512525),
 ('housing', 0.034673745873007096),
 ('cost living', 0.034558645706768726),
 ('median', 0.0341092208218325),
 ('apartment', 0.028312383664619207),
 ('home price', 0.026276175593222588),
 ('california', 0.025645654719534255),
 ('angeles', 0.022325803145221985)]

In [171]:
import plotly

In [170]:
topic_model.visualize_hierarchy()

### 상위 토픽 + 관련된 토픽 확인

- 위의 계층적 군집분석 결과를 보면 topic3과 topic21, topic0과 topic6이 묶임

In [126]:
topic_model.get_topic(0)

[('analyst refinitiv', 0.03372675178059123),
 ('analyst', 0.03358069195842215),
 ('cramer', 0.029788355301943913),
 ('price target', 0.02431798295350915),
 ('cent analyst', 0.02416615560766607),
 ('analyst factset', 0.01646362837866513),
 ('wall street', 0.01471902800971651),
 ('nvidia', 0.0134541032796249),
 ('refinitiv cent', 0.01326958911577641),
 ('trade alert', 0.013018680675714253)]

In [127]:
topic_model.get_topic(6)

[('iphone', 0.12257195997513622),
 ('apple', 0.106950452167445),
 ('ipad', 0.04105567881255938),
 ('smartphone', 0.040866859685472635),
 ('apple iphone', 0.035399444958679),
 ('huawei', 0.03232852302698514),
 ('device', 0.03137481122840396),
 ('tablet', 0.02255925250547163),
 ('android', 0.02098821779092229),
 ('iphone model', 0.019903119080530076)]

토픽 0, 6은 관련도가 낮은 토픽이므로, merge하지 않기로 함.

In [128]:
topic_model.get_topic(3)

[('pfizer', 0.03984946568083507),
 ('vaccine', 0.036666046213431495),
 ('pharmacy', 0.031354301864470226),
 ('medication', 0.031088154769561154),
 ('obesity', 0.02947804525251283),
 ('novo', 0.023910544694010386),
 ('cvs', 0.02173974013150382),
 ('telegram', 0.020354560583392255),
 ('marketplace', 0.020174584958086234),
 ('covid', 0.017711835197507627)]

In [129]:
topic_model.get_topic(21)

[('bric', 0.0945983815734802),
 ('russia', 0.08458820400459105),
 ('ukraine', 0.0673767073142626),
 ('ukraines', 0.03782669575290358),
 ('arabia', 0.03613214881783308),
 ('saudi arabia', 0.03613214881783308),
 ('moscow', 0.031989020757038834),
 ('president vladimir', 0.03146708884101227),
 ('china russia', 0.025810295868082356),
 ('war ukraine', 0.0210772127345319)]

토픽 3과 21이 ai 테마로 묶인다고 판단되어 두 토픽의 keyword를 추출하고 merge 후 토픽 share값을 확인함.

In [134]:
# 키워드 추출
topic_0_keyword = [tu[0] for tu in topic_model.get_topic(3)]
topic_21_keyword = [tu[0] for tu in topic_model.get_topic(21)]
keyword = topic_0_keyword + topic_21_keyword
print(keyword)

['pfizer', 'vaccine', 'pharmacy', 'medication', 'obesity', 'novo', 'cvs', 'telegram', 'marketplace', 'covid', 'bric', 'russia', 'ukraine', 'ukraines', 'arabia', 'saudi arabia', 'moscow', 'president vladimir', 'china russia', 'war ukraine']


In [135]:
# topic merge(ai, gpu)
topics_to_merge = [3, 21]
topic_model.merge_topics(all_texts, topics_to_merge)

In [136]:
#merge 후 share 값 확인
#토픽 3, 21을 합친 토픽이 가장 높은 share를 차지함
topic_stat = get_topic_stats(topic_model).head(20).set_index('Topic')
topic_stat

,Count,Share,CumulativeShare,Name,Representation
Topic,,,,,
-1,128,20.779221,20.779221,-1_bitcoin_resume_oracle_market,"[bitcoin, resume, oracle, market, sec, etf, ap..."
0,34,5.519481,26.298701,0_analyst refinitiv_price target_cent analyst_...,"[analyst refinitiv, price target, cent analyst..."
1,34,5.519481,31.818182,1_bric_russia_ukraine_pfizer,"[bric, russia, ukraine, pfizer, pharmacy, saud..."
2,29,4.707792,36.525974,2_ai_chatgpt_google_ai model,"[ai, chatgpt, google, ai model, openai, chatbo..."
3,28,4.545455,41.071429,3_retailer_walmart_foot locker_merchandise,"[retailer, walmart, foot locker, merchandise, ..."
4,24,3.896104,44.967532,4_cnn_disney_microsoft_activision,"[cnn, disney, microsoft, activision, espn, sal..."
5,23,3.733766,48.701299,5_election_president donald_indictment_case,"[election, president donald, indictment, case,..."
6,21,3.409091,52.110390,6_iphone_apple_smartphone_apple watch,"[iphone, apple, smartphone, apple watch, apple..."
7,20,3.246753,55.357143,7_playlist_schwartz_taylor_mindset,"[playlist, schwartz, taylor, mindset, billiona..."


In [137]:
# 8월 간 토픽 언급량 추이 시각화
topics_over_time = topic_model.topics_over_time(all_texts, timestamps,
                                                global_tuning=True, evolution_tuning=True, nr_bins=8)


8it [04:40, 35.06s/it]


In [138]:
#상위 5개 토픽
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=5)

등락이 크지 않고, 꾸준히 상승하고 있는 ai로 테마를 선정

In [139]:
#모델 저장
topic_model.save(os.path.join(PATH, "NH_topics_model"))

In [140]:
print(keyword)

['pfizer', 'vaccine', 'pharmacy', 'medication', 'obesity', 'novo', 'cvs', 'telegram', 'marketplace', 'covid', 'bric', 'russia', 'ukraine', 'ukraines', 'arabia', 'saudi arabia', 'moscow', 'president vladimir', 'china russia', 'war ukraine']


In [141]:
# 유의미한 키워드만 추출
best_keywords = ['ai', 'google', 'chatgpt', 'chatbot', 'openai', 'czech', 'ai model', 'language model', 'generative ai', 'aws', 'gpu', 'amd', 'processing unit', 'vmware', 'graphic processing', 'computing', 'ai model', 'micro device', 'advanced micro']

# 5. 언급량이 많은 토픽 기반 기업 찾기

## 데이터 전처리(복수형, 동의어)

In [142]:
!pip install inflect
!pip install nltk

In [143]:
import pandas as pd
from nltk.corpus import wordnet as wn
import nltk
import inflect


nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [144]:
print(best_keywords)

['ai', 'google', 'chatgpt', 'chatbot', 'openai', 'czech', 'ai model', 'language model', 'generative ai', 'aws', 'gpu', 'amd', 'processing unit', 'vmware', 'graphic processing', 'computing', 'ai model', 'micro device', 'advanced micro']


In [145]:
combined_list = []

for word in best_keywords:
    lower_word = word.lower()
    synsets = wn.synsets(lower_word)
    if synsets:
        first_synset = synsets[0]
        combined_list.extend(first_synset.lemma_names())
    combined_list.append(word)

# 결과 리스트 출력
combined_list = list(set([word for word in combined_list if "_" not in word]))
print(combined_list)

['AI', 'openai', 'ai model', 'chatgpt', 'processing unit', 'advanced micro', 'generative ai', 'language model', 'AMD', 'micro device', 'graphic processing', 'Czech', 'aws', 'google', 'chatbot', 'computing', 'vmware', 'gpu', 'Google', 'ai', 'amd', 'czech']


In [146]:
#복수형 생성
def singular_to_plural(word):
    p = inflect.engine()
    return p.plural(word)

new_list = []

for word in combined_list:
    new_word = singular_to_plural(word)
    new_list.append(new_word)

all_list = list(set(combined_list + new_list))

In [147]:
# 최종 키워드
print(all_list)

['AI', 'openai', 'micro devices', 'ai model', 'vmwares', 'chatgpt', 'ais', 'processing unit', 'generative ai', 'chatgpts', 'googles', 'chatbots', 'Czechs', 'AMDS', 'language model', 'advanced micros', 'micro device', 'AMD', 'AIS', 'graphic processings', 'graphic processing', 'computings', 'language models', 'Czech', 'aws', 'google', 'chatbot', 'computing', 'vmware', 'Googles', 'gpu', 'ai models', 'processing units', 'aw', 'Google', 'generative ais', 'czech', 'amds', 'czechs', 'openais', 'gpus', 'ai', 'amd', 'advanced micro']


## 토픽 키워드와 연관된 기업 찾기

In [148]:
# 기업 description 불러오기(yfinance crawling)
stock_info_df = pd.read_csv(os.path.join(READ_PATH, 'stock_description.csv'))

In [149]:
# 토픽 관련 기업 찾기
def retrieve_companies_by_keywords(keywords):
    keywords_set = set([word.lower() for word in keywords])

    # 기업 리스트
    cp = []

    for i, row in stock_info_df.iterrows():
        description = row['description']

        #단어 추출
        if isinstance(description, str):
            description_words = set(description.replace(",", '').lower().replace('.', '').split(" "))

            # 토픽 키워드와 description이 겹치는 기업 찾기
            if description_words & keywords_set:
                cp.append(row['tck_iem_cd'])

    return list(set(cp))

In [150]:
len(retrieve_companies_by_keywords(all_list))

100

In [151]:
# 키워드 언급횟수 count
def count_companies_by_keywords(keywords):
    keywords_set = set([word.lower() for word in keywords])

    # 키워드 언급횟수 딕셔너리 생성
    keyword_counts = {word: 0 for word in keywords_set}

    for i, row in stock_info_df.iterrows():
        description = row['description']

        if isinstance(description, str):
            description_words = set(description.replace(",", '').lower().replace('.', '').split(" "))

            # 키워드 횟수 카운트
            for word in keywords_set:
                if word in description_words:
                    keyword_counts[word] += 1

    for keyword, count in keyword_counts.items():
        print(f"{keyword}: {count} companies")

In [152]:
count_companies_by_keywords(all_list)

openai: 0 companies
micro devices: 0 companies
ai model: 0 companies
vmwares: 0 companies
chatgpt: 1 companies
ais: 0 companies
processing unit: 0 companies
advanced micro: 0 companies
generative ai: 0 companies
chatgpts: 0 companies
googles: 0 companies
chatbots: 0 companies
language model: 0 companies
advanced micros: 0 companies
micro device: 0 companies
graphic processings: 0 companies
graphic processing: 0 companies
computings: 0 companies
language models: 0 companies
aws: 2 companies
google: 9 companies
chatbot: 0 companies
computing: 52 companies
vmware: 0 companies
gpu: 1 companies
ai models: 0 companies
processing units: 0 companies
aw: 0 companies
generative ais: 0 companies
amds: 0 companies
czechs: 0 companies
openais: 0 companies
gpus: 1 companies
ai: 40 companies
amd: 3 companies
czech: 4 companies


In [153]:
cp_list = retrieve_companies_by_keywords(all_list)

In [154]:
#토픽과 관련있는 기업 추출
print(cp_list)

['LKCO', 'TTMI', 'MPWR', 'EXAI', 'ABSI', 'SCSC', 'LKQ', 'GFAI', 'RGTI', 'LSCC', 'RTC', 'SOUN', 'AOSL', 'NEWT', 'ANSS', 'DRS', 'MLGO', 'ECX', 'KOSS', 'NTGR', 'DIOD', 'API', 'THRM', 'CCCS', 'SGH', 'CEVA', 'GRRR', 'OTEX', 'KOD', 'HUT', 'NTAP', 'KC', 'REKR', 'WETG', 'GOOG', 'AVPT', 'PLTK', 'CD', 'PRST', 'FWRG', 'RBBN', 'LNTH', 'LVOX', 'VOD', 'FFIV', 'AUUD', 'XRX', 'MARK', 'CSCO', 'RDFN', 'INTC', 'CRNC', 'VRNT', 'AKAM', 'WDC', 'MSFT', 'INTA', 'APLD', 'AMPG', 'MCLD', 'CDNS', 'SASI', 'NUWE', 'NNOX', 'RNLX', 'RIOT', 'VERI', 'LWLG', 'CNXA', 'GLMD', 'PALT', 'AEHR', 'NSIT', 'XNET', 'OCGN', 'CTSH', 'AMKR', 'ZBRA', 'SANM', 'PERI', 'STCN', 'AIMD', 'MCHP', 'SMCI', 'AMD', 'NICE', 'KTOS', 'QUBT', 'EGIO', 'VUZI', 'ALTR', 'NVDA', 'QMCO', 'TASK', 'POAI', 'BRQS', 'LIZI', 'TXN', 'BELFB', 'TROO']


In [155]:
len(cp_list)

100

### NASDAQ 8월 데이터에서 5번 이상 등장한 기업 필터링

In [156]:
#주식코드를 입력하면 NASDAQ 데이터프레임에서 관련 기사를 찾아주는 함수
def find_rows_by_tck(nasdaq_df, stock_code):
    rows_to_append = []

    for i in range(len(nasdaq_df)):
        lst = nasdaq_df['all_tck_iem_cd'].iloc[i]
        lst = lst.replace('[', '')
        lst = lst.replace(']', '')
        lst = lst.replace("'", '')
        lst = lst.replace(" ", '')
        lst = lst.split(',')

        if stock_code in lst:
            a = nasdaq_df.iloc[i]
            rows_to_append.append(a)

    if rows_to_append:
        nasdaq_rows = pd.concat(rows_to_append, axis=1).T.reset_index(drop=True)
        return nasdaq_rows
    else:
        return pd.DataFrame()

In [165]:
august_nasdaq_df = nasdaq_df[nasdaq_df['rgs_dt']>=20230801]

In [167]:
cp_list_filtered = []
for company in cp_list:
    # Find rows corresponding to the company using the provided function
    company_rows = find_rows_by_tck(august_nasdaq_df, company)

    # Count the number of rows
    row_count = len(company_rows)

    # Filter companies with at least 5 rows
    if row_count >= 5:
        cp_list_filtered.append(company)

In [169]:
len(cp_list_filtered)

58

# 6. XGBoost를 이용한 재무제표 정보 기반 기업 고르기
- XGBoost를 사용한 이유 : 해당 데이터에는 결측치가 많아 결측치를 효과적으로 처리하는 모델이 필요함. 또한 분류 모델 중 가장 성능이 높다고 알려진 XGBoost로 해당 분류를 진행함.
- 재무제표 데이터를 활용해 8월 대비 9월의 평균 주가가 오를 것으로 예상되는 기업을 추출함.

## 재무제표 정보 크롤링
- yfinance의 ticker.info 객체를 활용하여 가장 최근의 재무제표 정보를 가져오는데, 최초 크롤링 시점이 9월로, 크롤링 결과가 달라질 수 있다는 점 유의해야 함

In [1]:
import yfinance as yf
import pandas as pd
import warnings
import os
warnings.filterwarnings("ignore")

In [6]:
READ_PATH = '/content/drive/MyDrive/final_data/'
PATH = '/content/'

In [7]:
stock = pd.read_csv(os.path.join(READ_PATH, 'NASDAQ_FC_STK_IEM_IFO.csv'), encoding = 'cp949')
stock

,isin_cd,tck_iem_cd,fc_sec_krl_nm,fc_sec_eng_nm
0,US00211V1061,AACG,ATA ...,ATA CreatGlo ...
1,US00032Q1040,AADI,Aadi Bioscience ...,Aadi Bioscience ...
2,US02376R1023,AAL,아메리칸 에어라인스 그룹 ...,American Airline ...
3,US03823U1025,AAOI,어플라이드 옵토일렉트로닉스 ...,AOI ...
4,US0003602069,AAON,에이에이온 ...,AAON ...
...,...,...,...,...
2738,US4884452065,ZVRA,Zevra ...,Zevra ...
2739,US98987D1028,ZVSA,Zyversa ...,Zyversa ...
2740,US98985Y1082,ZYME,Zymeworks ...,Zymeworks ...
2741,US98986X1090,ZYNE,자이너바 파마수티컬스 ...,Zynerba Pharms ...


In [8]:
# 'tck_iem_cd' 열의 공백 제거
stock['tck_iem_cd'] = stock['tck_iem_cd'].str.strip()

In [9]:
# 종목 정보 추가 함수
def add_stock_info(df):
    info_columns_mapper = {

        # 정보 관련
        'sector': '섹터',
        'industry': '산업군',
        'longBusinessSummary' : 'description',

        # 매매 정보 관련
        'sharesOutstanding': '발행주식수',
        'averageVolume10days': '종목평균거래량(10일)',
        'averageVolume': '종목평균거래량',
        'heldPercentInstitutions': '기관보유비율',
        'shortRatio': '일일공매도비율',
        'sharesPercentSharesOut': '발행주식대비공매도비율',
        'shortPercentOfFloat': '유동주식중공매도비율',

        # 가격 관련
        'marketCap': '시가총액',  # 200B:mega // 10B~200B:large // 2B-10B:medium // 300M~2B:small // 50M~300M:micro // ~50M:nano
        'currentPrice': '현재가',
        'fiftyDayAverage': '50일평균가',
        'twoHundredDayAverage': '200일평균가',
        'fiftyTwoWeekHigh': '52주최고가',
        'fiftyTwoWeekLow': '52주최저가',
        'SandP52WeekChange': 'S&P_52주변동성',
        '52WeekChange': '52주변동성',
        'ytdReturn': '연초대비수익률',
        'fiveYearAverageReturn': '5년연평균수익률',  # 5년연평균수익률
        'beta': '베타값',  # 5년 데이터, 개별주식의 변동률을 의미. 1에 가까울수록 시장과 가깝고, 1을 넘어가면 시장 대비 고변동, 0으로 가까우면 시장 대비 저변동 주식을 의미함.

        # 현금 창출, 매출 관련 (ttm)
        'totalRevenue': '총매출액',
        'grossProfits': '매출총이익',  # 매출이익(매출액 - 매출원가)
        'revenuePerShare': '주당매출액',
        'ebitda': 'EBITDA',  # 감가상각 등의 부가비용을 차감하기 전의 금액, 영업 활동을 통한 현금 창출 능력. 유형자산의 가치까지 포함하는 지표
        'ebitdaMargins': 'EBITDA마진',  # 유형자산의 유지비용을 고려한 기업의 현금 창출 능력

        # 재무 상태 관련 (mrq)
        'debtToEquity': '부채자본비율',
        'operatingCashflow': '영업현금흐름',  # 영업현금흐름 : 영업이익 - 법인세 - 이자비용 + 감가상각비
        'freeCashflow': '잉여현금흐름',  # 기업의 본원적 영업활동을 위해 현금을 창출하고, 영업자산에 투자하고도 남은 현금
        'totalCashPerShare': '주당현금흐름',
        'currentRatio': '유동비율',  # 회사가 가지고 있는 단기 부채 상환 능력
        'quickRatio': '당좌비율',  # 회사가 가지고 있는 단기 부채 상환 능력
        'overallRisk': '위험 점수',

        # 경영 효율 관련
        'returnOnAssets': '자기자본이익률',  # mrq : 간단히 말해, 얼마를 투자해서 얼마를 벌었냐
        'returnOnEquity': '총자산순이익률',  # mrq : ROE와 비교하여 기업이 가지고 있는 부채의 비중을 볼 때
        'grossMargins': '매출총이익률',  # ttm : 매출이익(매출액 - 매출원가) / 매출액 : 매출이익률, Gross Profit Margin (GPM)
        'operatingMargins': '영업이익률',  # ttm : 매출총이익 - 판관비 - 감가상각비
        'profitMargins': '순이익률',  # ttm : Net Income(순이익) / Revenue(총수익) : 순이익률, Net Profit Margin (NPM)

        # 기업 자산 관련
        'totalCash': '총현금액',
        'totalDebt': '총부채액',

        # 기업 가치 관련
        'priceToBook': 'PBR',  # 기업이 가진 순 자산에 비해 주가가 얼마나 비싼지
        'enterpriseValue': '기업가치',  # 기업가치 : 시가총액 + (총차입금 - 현금성 자산)
        'enterpriseToRevenue': 'EV/R',  # 매출액대비 기업가치 비율
        'enterpriseToEbitda': 'EV/EBITDA',  # EBITDA대비 기업가치 비율 : PER과 의미적으로 비슷한 지표
        'forwardEps': '선행1년EPS',  # 주당순이익, 보통 5년동안의 EPS를 관찰해서 추이를 봄
        'trailingEps': '1년EPS',  # 주당순이익 = 당기순이익 / 유통주식수
        'priceToSalesTrailing12Months': '1년PSR',  # 주가매출액비율 (1년 기준)
        'forwardPE': '선행1년PER',  # 향후 1년동안 예상되는 PER
        'trailingPE': '1년PER',  # 현재 PER. 기업이 한 주당 벌어들이는 순이익에 비해, 실제 주가가 몇 배가 되는 지 나타내는 지표. 고평가 저평가에 사용

        # 성장성 관련
        'revenueGrowth': 'mrq매출액증가율',
        'earningsGrowth': 'mrq수익상승률',
        'earningsQuarterlyGrowth': 'yoy수익상승률',  # yoy : 지난해 동일 분기 대비 최근 분기의 수익 상승률
        'revenueQuarterlyGrowth': 'yoy매출상승률',  # yoy : 지난해 동일 분기 대비 최근 분기의 매출 상승률
        'heldPercentInsiders': '직원보유비율',

        'Research Development': 'R&D비용',
        'Net Income': '순이익',
        'Gross Profit': '매출총이익',
        'Operating Income': '영업이익',
        'Total Revenue': '총매출',
        'Cost Of Revenue': '제품원가',

        'Research Development': 'R&D비용',
        'Net Income': '순이익',
        'Gross Profit': '매출총이익',
        'Operating Income': '영업이익',
        'Total Revenue': '총매출',
        'Cost Of Revenue': '제품원가',

        'Total Liab': '총부채',
        'Total Stockholder Equity': '자기자본',
        'Total Assets': '총자산',
    }

    for i, row in df.iterrows():
        tck_iem_cd = row['tck_iem_cd']  # 종목티커코드 가져오기
        ticker = yf.Ticker(tck_iem_cd)  # 종목티커코드로 Ticker 객체 생성

        # Ticker 객체를 통해 종목 정보 가져오기
        stock_info = ticker.info

        # 필요한 정보만 추출하여 데이터프레임에 추가
        for key, value in info_columns_mapper.items():
            if key in stock_info:
                df.at[i, value] = stock_info[key]

    return df


In [ ]:
stock_info = add_stock_info(stock)

stock_info

In [ ]:
stock_info.isnull().sum()

In [ ]:
stock_info.to_csv(os.join.path(PATH, 'stock_info.csv'), index = False)

## 데이터 전처리 및 모델 학습

In [11]:
!pip install xgboost

In [20]:
import pandas as pd
import numpy as np
import yfinance as yf

import xgboost as xgb
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 경고 메시지를 숨길 설정 추가
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [21]:
df = pd.read_csv(os.path.join(READ_PATH, 'stock_info.csv'))
df.head(5)

,isin_cd,tck_iem_cd,fc_sec_krl_nm,fc_sec_eng_nm,섹터,산업군,description,발행주식수,종목평균거래량(10일),종목평균거래량,...,mrq매출액증가율,직원보유비율,유동주식중공매도비율,매출총이익,영업현금흐름,잉여현금흐름,위험 점수,1년PER,mrq수익상승률,yoy수익상승률
0,US00211V1061,AACG,ATA ...,ATA CreatGlo ...,Consumer Defensive,Education & Training Services,"ATA Creativity Global, together with its subsi...",31585900.0,3800.0,6822.0,...,0.065,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US00032Q1040,AADI,Aadi Bioscience ...,Aadi Bioscience ...,Healthcare,Biotechnology,"Aadi Bioscience, Inc., a clinical-stage biopha...",24519900.0,130880.0,131912.0,...,0.804,0.12510,0.0387,-1.878100e+07,-5.757300e+07,-3.674612e+07,8.0,NaN,NaN,NaN
2,US02376R1023,AAL,아메리칸 에어라인스 그룹 ...,American Airline ...,Industrials,Airlines,"American Airlines Group Inc., through its subs...",653361984.0,27378180.0,24171950.0,...,0.047,0.01303,0.1125,1.144300e+10,4.345000e+09,1.205875e+09,7.0,3.324742,1.769,1.811
3,US03823U1025,AAOI,어플라이드 옵토일렉트로닉스 ...,AOI ...,Technology,Communication Equipment,"Applied Optoelectronics, Inc. designs, manufac...",33342500.0,2753250.0,4239232.0,...,-0.204,0.08457,0.1936,3.362700e+07,-6.804000e+06,1.881688e+07,NaN,NaN,NaN,NaN
4,US0003602069,AAON,에이에이온 ...,AAON ...,Industrials,Building Products & Equipment,"AAON, Inc., together with its subsidiaries, en...",81609504.0,547790.0,434956.0,...,0.360,0.17443,0.0365,2.375720e+08,1.228380e+08,1.345638e+07,4.0,31.464090,1.734,1.865


In [22]:
#사용되지 않는 컬럼 drop
df.drop(['isin_cd', 'fc_sec_krl_nm', 'fc_sec_eng_nm', '섹터', '산업군', 'description', '위험 점수', 'S&P_52주변동성', '52주변동성', '1년PER', 'mrq수익상승률', 'yoy수익상승률'], axis = 1, inplace=True)

In [23]:
df.isnull().sum()

tck_iem_cd        0
발행주식수            69
종목평균거래량(10일)      2
종목평균거래량           2
기관보유비율           97
일일공매도비율          26
발행주식대비공매도비율      86
시가총액             63
현재가              18
50일평균가            2
200일평균가           2
52주최고가            0
52주최저가            0
베타값             221
총매출액            327
주당매출액           329
EBITDA          384
EBITDA마진         25
부채자본비율          555
주당현금흐름           33
유동비율            237
당좌비율            238
자기자본이익률          52
총자산순이익률         222
매출총이익률           25
영업이익률            25
순이익률             25
총현금액             31
총부채액             30
PBR             286
기업가치             28
EV/R            347
EV/EBITDA       386
선행1년EPS         340
1년EPS            30
1년PSR           371
선행1년PER         342
mrq매출액증가율       473
직원보유비율           97
유동주식중공매도비율      228
매출총이익           673
영업현금흐름           87
잉여현금흐름          356
dtype: int64

In [24]:
df.describe()

,발행주식수,종목평균거래량(10일),종목평균거래량,기관보유비율,일일공매도비율,발행주식대비공매도비율,시가총액,현재가,50일평균가,200일평균가,...,선행1년EPS,1년EPS,1년PSR,선행1년PER,mrq매출액증가율,직원보유비율,유동주식중공매도비율,매출총이익,영업현금흐름,잉여현금흐름
count,2.674000e+03,2.741000e+03,2.741000e+03,2646.000000,2717.000000,2657.000000,2.680000e+03,2725.000000,2741.000000,2741.000000,...,2403.000000,2713.000000,2.372000e+03,2401.000000,2270.000000,2646.000000,2515.000000,2.070000e+03,2.656000e+03,2.387000e+03
mean,1.264070e+08,1.615812e+06,1.441136e+06,0.514881,5.350258,0.035366,9.043334e+09,31.025605,32.849842,31.799264,...,0.870337,-0.238580,inf,inf,1.916444,0.154126,0.052575,2.039863e+09,5.953435e+08,3.464925e+08
std,5.021737e+08,8.746842e+06,7.425701e+06,0.355742,6.825366,0.044332,9.317390e+10,95.969782,98.009973,86.931095,...,8.833949,79.040184,NaN,NaN,44.536094,0.191754,0.067874,1.849851e+10,5.040937e+09,3.692044e+09
min,0.000000e+00,1.170000e+03,2.720000e+03,0.000000,0.000000,0.000000,2.918270e+05,0.021000,0.061180,0.154255,...,-252.040000,-1097.750000,-2.592637e+04,-9506.000000,-0.999000,0.000000,0.000000,-3.123000e+09,-9.591814e+09,-5.558650e+10
25%,1.817798e+07,5.773000e+04,7.365900e+04,0.162888,1.400000,0.007500,4.486497e+07,1.610000,2.048480,2.541350,...,-0.850000,-1.630000,6.580895e-01,-2.412698,-0.117000,0.019547,0.011000,1.162800e+07,-3.026525e+07,-2.986520e+07
50%,4.379250e+07,2.545100e+05,2.780500e+05,0.538950,3.450000,0.020000,3.046661e+08,7.240000,8.304600,8.992050,...,0.090000,-0.390000,2.062760e+00,4.562791,0.045000,0.074030,0.028900,1.390690e+08,-2.567187e+06,-3.829375e+06
75%,1.011508e+08,8.514300e+05,8.477980e+05,0.840048,6.670000,0.045600,1.686507e+09,25.240000,26.801600,27.380050,...,1.990000,1.190000,5.369715e+00,13.941175,0.240000,0.217095,0.067250,5.933670e+08,1.098803e+08,4.674856e+07
max,1.563420e+10,1.965027e+08,2.552021e+08,2.630500,93.590000,0.397300,2.732702e+12,3062.540000,3066.063700,2623.915500,...,180.990000,3416.730000,inf,inf,2026.887000,1.701160,0.913300,6.858540e+11,1.130720e+11,9.068050e+10


In [25]:
# 데이터프레임 내의 모든 무한대 값을 결측치로 변경
df = df.replace([np.inf, -np.inf], np.nan)

In [27]:
stock = pd.read_csv(os.path.join(READ_PATH, 'NASDAQ_DT_FC_STK_QUT.csv'))
stock

,trd_dt,tck_iem_cd,gts_iem_ong_pr,gts_iem_hi_pr,gts_iem_low_pr,gts_iem_end_pr,gts_acl_trd_qty,gts_sll_cns_sum_qty,gts_byn_cns_sum_qty
0,20230103,NVDA,148.5100,149.9600,140.9600,143.1500,40127658.0,0.0,0.0
1,20230103,APLT,0.7535,0.7900,0.7308,0.7522,63714.0,19658.0,44056.0
2,20230103,ANY,1.8900,2.0293,1.8354,1.9600,24896.4,0.0,0.0
3,20230103,CLRB,1.6600,1.7200,1.6300,1.6300,21212.0,16444.0,4768.0
4,20230103,NYMTM,18.4300,18.7400,18.4300,18.7000,8061.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
455333,20230830,BLZE,5.9900,6.1290,5.8300,5.8600,443470.0,205162.0,238308.0
455334,20230830,ELDN,1.4800,1.5794,1.4100,1.4900,13462.0,5492.0,7970.0
455335,20230830,BMBL,15.9000,16.4400,15.7900,16.3200,1030250.0,576506.0,453744.0
455336,20230830,PLRX,16.7900,16.7900,16.2250,16.7100,319531.0,138554.0,180977.0


In [28]:
# 2023년 8월 데이터만 선택
Aug_df = stock[stock['trd_dt'] >= 20230801]
Aug_df

,trd_dt,tck_iem_cd,gts_iem_ong_pr,gts_iem_hi_pr,gts_iem_low_pr,gts_iem_end_pr,gts_acl_trd_qty,gts_sll_cns_sum_qty,gts_byn_cns_sum_qty
394992,20230801,HTCR,1.48,1.4800,1.330,1.35,32901.0,18224.0,14677.0
394993,20230801,INTU,510.56,510.5600,502.830,508.50,1495968.0,1126433.0,500009.0
394994,20230801,PETZ,1.29,1.3400,1.240,1.30,9357.0,2139.0,7218.0
394995,20230801,AXON,184.45,186.0499,183.260,183.33,425596.0,164982.0,260614.0
394996,20230801,CGABL,17.98,18.0320,17.855,17.98,15779.0,7741.0,8038.0
...,...,...,...,...,...,...,...,...,...
455333,20230830,BLZE,5.99,6.1290,5.830,5.86,443470.0,205162.0,238308.0
455334,20230830,ELDN,1.48,1.5794,1.410,1.49,13462.0,5492.0,7970.0
455335,20230830,BMBL,15.90,16.4400,15.790,16.32,1030250.0,576506.0,453744.0
455336,20230830,PLRX,16.79,16.7900,16.225,16.71,319531.0,138554.0,180977.0


In [29]:
#기업별 8월의 종가 평균 계산
close_Augs = Aug_df.groupby(['tck_iem_cd'])['gts_iem_end_pr'].mean().reset_index()
close_Augs.columns = ['tck_iem_cd', 'Aug_avg_close']
close_Augs

,tck_iem_cd,Aug_avg_close
0,AACG,1.258545
1,AADI,6.030909
2,AAL,15.391818
3,AAOI,12.708636
4,AAON,63.469859
...,...,...
2738,ZVRA,5.050455
2739,ZVSA,0.157705
2740,ZYME,7.491818
2741,ZYNE,0.917773


### 9월 종가 평균 데이터 생성(yfinance 주가 데이터 크롤링)

In [ ]:
# 9월 주가 데이터를 저장할 데이터 프레임 생성
close_Seps = pd.DataFrame(columns=['tck_iem_cd', 'Sep_avg_close'])

# df['tck_iem_cd']에 있는 각 티커 코드에 대해 반복
for ticker in df['tck_iem_cd']:
    stock = yf.Ticker(ticker)
    start_date = "2023-09-01"
    end_date = "2023-09-30"
    data = stock.history(start=start_date, end=end_date)

    # 해당 티커 코드의 8월 주가 데이터 평균 계산
    close_Sep = data['Close'].mean()

    # 결과를 데이터 프레임에 추가
    close_Seps = close_Seps.append({'tck_iem_cd': ticker, 'Sep_avg_close': close_Sep}, ignore_index=True)

In [ ]:
close_Seps

In [ ]:
avg_close = pd.merge(close_Augs, close_Seps, on='tck_iem_cd', how='inner')
avg_close

In [ ]:
# 9월 종가 평균이 8월 종가 평균 보다 높으면 1, 낮으면 0으로 설정
# 1 = 주가가 오른 기업, 0 = 주가가 떨어진 기업
avg_close['label'] = avg_close['Aug_avg_close'] < avg_close['Sep_avg_close']

# 결과를 0 또는 1로 변환
avg_close['label'] = avg_close['label'].astype(int)
avg_close['label']

In [ ]:
avg_close['label'].value_counts()

In [ ]:
avg_close.to_csv(os.path.join(PATH, "avg_close.csv"), index=False)

In [34]:
avg_close = pd.read_csv(os.path.join(READ_PATH, 'avg_close.csv'))


In [35]:
# 모델링에 필요한 data 준비
y = avg_close['label']
df = pd.concat([df, y], axis = 1)
df

,tck_iem_cd,발행주식수,종목평균거래량(10일),종목평균거래량,기관보유비율,일일공매도비율,발행주식대비공매도비율,시가총액,현재가,50일평균가,...,1년EPS,1년PSR,선행1년PER,mrq매출액증가율,직원보유비율,유동주식중공매도비율,매출총이익,영업현금흐름,잉여현금흐름,label
0,AACG,31585900.0,3800.0,6822.0,0.16365,0.12,0.0000,3.600792e+07,1.1400,1.27040,...,-0.19,0.172124,-1.151515,0.065,0.00000,NaN,NaN,NaN,NaN,0
1,AADI,24519900.0,130880.0,131912.0,0.60698,4.61,0.0254,1.223543e+08,4.9900,5.81770,...,-2.31,5.680066,-1.708904,0.804,0.12510,0.0387,-1.878100e+07,-5.757300e+07,-3.674612e+07,0
2,AAL,653361984.0,27378180.0,24171950.0,0.58465,2.89,0.0991,8.428369e+09,12.9000,15.44900,...,3.88,0.159345,4.607143,0.047,0.01303,0.1125,1.144300e+10,4.345000e+09,1.205875e+09,0
3,AAOI,33342500.0,2753250.0,4239232.0,0.30670,1.06,0.1843,3.194212e+08,9.5800,11.05650,...,-2.28,1.500172,23.949999,-0.204,0.08457,0.1936,3.362700e+07,-6.804000e+06,1.881688e+07,0
4,AAON,81609504.0,547790.0,434956.0,0.71752,5.59,0.0320,4.647662e+09,56.9500,64.40980,...,1.81,4.438548,25.311111,0.360,0.17443,0.0365,2.375720e+08,1.228380e+08,1.345638e+07,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2738,ZVRA,36207100.0,151370.0,153025.0,0.19795,19.27,0.0681,1.759665e+08,4.8600,5.01870,...,-0.92,10.637560,-7.043478,5.515,0.11387,0.0685,1.011500e+07,-2.380900e+07,-1.657262e+07,1
2739,ZVSA,36394200.0,21279420.0,4787927.0,0.27967,0.39,0.0093,4.822231e+06,0.1325,0.16302,...,-7.32,NaN,-0.236607,NaN,0.54438,0.0110,NaN,-9.196134e+06,-5.211207e+06,0
2740,ZYME,67826704.0,300920.0,484875.0,0.96483,8.96,0.0732,4.340909e+08,6.4000,7.38320,...,2.73,0.969593,-4.183007,0.287,0.04877,0.0866,2.037790e+08,1.647550e+08,8.823938e+07,0
2741,ZYNE,53939400.0,786110.0,886206.0,0.12127,0.15,0.0045,7.227880e+07,1.3400,0.89674,...,-0.80,NaN,-2.126984,NaN,0.06070,0.0048,NaN,-3.164196e+07,-1.878967e+07,1


In [36]:
# 토픽모델링 후, 선정된 테마에서 나온 기업 test set으로 생성(추후 확률값을 concat하기 위해 미리 index 재지정)
theme_tickers = cp_list_filtered

train = df[~df['tck_iem_cd'].isin(theme_tickers)].reset_index(drop=True)
test = df[df['tck_iem_cd'].isin(theme_tickers)].reset_index(drop=True)

print(train.shape)
print(test.shape)

(2685, 44)
(58, 44)


In [37]:
test.head()

,tck_iem_cd,발행주식수,종목평균거래량(10일),종목평균거래량,기관보유비율,일일공매도비율,발행주식대비공매도비율,시가총액,현재가,50일평균가,...,1년EPS,1년PSR,선행1년PER,mrq매출액증가율,직원보유비율,유동주식중공매도비율,매출총이익,영업현금흐름,잉여현금흐름,label
0,AEHR,2.876250e+07,630070.0,1071927.0,0.68129,6.13,0.1917,1.227008e+09,42.66,47.4692,...,0.47,18.888384,28.630873,0.098,0.07064,0.2086,NaN,1.001100e+07,3.546375e+06,1
1,AKAM,1.517130e+08,2452820.0,1638212.0,0.94811,4.34,0.0520,1.631673e+10,107.55,99.9558,...,3.03,4.456791,16.752337,0.036,0.01811,0.0602,2.237817e+09,1.310604e+09,6.953220e+08,1
2,ALTR,5.407280e+07,198760.0,230332.0,0.99869,10.39,0.0343,5.058653e+09,62.27,66.9991,...,-0.55,8.618116,54.147827,0.064,0.01465,0.0650,4.493320e+08,1.107580e+08,1.166884e+08,0
3,AMD,1.615670e+09,50847200.0,60425574.0,0.74466,0.42,0.0174,1.554275e+11,96.20,108.9964,...,-0.04,7.104930,23.180721,-0.182,0.00477,0.0175,1.205100e+10,2.397000e+09,3.586375e+09,0
4,AMKR,2.457630e+08,1897850.0,1146295.0,0.38818,4.57,0.0142,5.502633e+09,22.39,26.5522,...,2.35,0.795252,8.780392,-0.031,0.59522,0.0528,1.329987e+09,1.059065e+09,4.633750e+06,0


In [38]:
X_train = train.drop(['tck_iem_cd', 'label'], axis = 1)
y_train = train['label']

X_test = test.drop(['tck_iem_cd', 'label'], axis = 1)
y_test = test['label']

In [39]:
#train set 스케일링
from sklearn.preprocessing import MinMaxScaler

# Min-Max 스케일러 생성
scaler = MinMaxScaler()

# 스케일러를 Train 데이터에 맞춥니다.
scaler.fit(X_train)

# Train 데이터를 스케일링합니다.
X_train = scaler.transform(X_train)

In [40]:
from imblearn.over_sampling import RandomOverSampler

# train 데이터를 train과 validation 세트로 분리
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# 오버샘플링 수행
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)


In [42]:
from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [300, 400, 500],
    'max_depth': [5, 8, 10],
    'learning_rate': [0.05, 0.1, 0.2]
}

# Grid Search를 사용한 모델 초기화
model = xgb.XGBClassifier(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Grid Search로 최적의 하이퍼파라미터 찾기
grid_search.fit(X_train_resampled, y_train_resampled)

# 최적의 모델과 하이퍼파라미터 출력
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print("최적의 하이퍼파라미터:", best_params)

최적의 하이퍼파라미터: {'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 500}


In [43]:
# Validation 세트에서 모델 성능 확인
y_valid_pred = best_model.predict(X_valid)
print("Validation 세트 정확도:", accuracy_score(y_valid, y_valid_pred))
print("Validation 세트 리포트:\n", classification_report(y_valid, y_valid_pred))

# Test 세트에서 모델 성능 확인
y_test_pred = best_model.predict(X_test)
print("Test 세트 정확도:", accuracy_score(y_test, y_test_pred))
print("Test 세트 리포트:\n", classification_report(y_test, y_test_pred))

Validation 세트 정확도: 0.7839851024208566
Validation 세트 리포트:
               precision    recall  f1-score   support

           0       0.82      0.93      0.87       420
           1       0.51      0.27      0.36       117

    accuracy                           0.78       537
   macro avg       0.66      0.60      0.61       537
weighted avg       0.75      0.78      0.76       537

Test 세트 정확도: 0.6724137931034483
Test 세트 리포트:
               precision    recall  f1-score   support

           0       0.78      0.72      0.75        40
           1       0.48      0.56      0.51        18

    accuracy                           0.67        58
   macro avg       0.63      0.64      0.63        58
weighted avg       0.69      0.67      0.68        58



In [44]:
# Test 세트에서 모델의 클래스 확률 예측
y_test_pred_proba = best_model.predict_proba(X_test)

# 해당 기업이 9월에 주가가 상승할 확률 추출
class_1_proba = y_test_pred_proba[:, 1]

# 결과 출력
print("해당 기업이 9월에 주가가 상승할 확률:")
Sep_prob = pd.DataFrame(class_1_proba, columns=['Sep_prob'])
Sep_prob.head()

해당 기업이 9월에 주가가 상승할 확률:


,Sep_prob
0,0.463278
1,0.334518
2,0.322709
3,0.487412
4,0.321548


In [55]:
# 최근 주목받는 테마에 속하는 기업과 해당 기업이 9월에 주가가 오를 확률에 대한 데이터프레임 생성
theme_financial_prob = pd.concat([test['tck_iem_cd'], Sep_prob], axis=1, ignore_index=True)
theme_financial_prob.columns = ['Company', 'xgboost_prob']

In [56]:
theme_financial_prob.to_csv(os.path.join(PATH, "theme_financial_prob.csv"), index=False)

### 8월 대비 9월 주가 상승 확률 기반 기업 필터링

In [80]:
#상승 확률이 상위 50%인 필터링함
financial_filtered_cp_df = theme_financial_prob.sort_values(by="xgboost_prob", ascending=False).head(len(theme_financial_prob)//2)

In [81]:
financial_filtered_cp_df

,Company,xgboost_prob
52,VOD,0.897710
22,INTC,0.847029
33,NICE,0.794178
17,DRS,0.750374
39,PERI,0.692716
27,LNTH,0.692540
14,CSCO,0.686896
20,GOOG,0.664316
47,SOUN,0.661067
25,KTOS,0.639091


In [83]:
financial_filtered_cp_list = financial_filtered_cp_df['Company'].tolist()
financial_filtered_cp_list

['VOD',
 'INTC',
 'NICE',
 'DRS',
 'PERI',
 'LNTH',
 'CSCO',
 'GOOG',
 'SOUN',
 'KTOS',
 'AOSL',
 'NEWT',
 'CCCS',
 'APLD',
 'MSFT',
 'CTSH',
 'RGTI',
 'PLTK',
 'WDC',
 'SCSC',
 'NTAP',
 'MCHP',
 'VUZI',
 'DIOD',
 'VRNT',
 'AMD',
 'CEVA',
 'XRX',
 'AEHR']

# 7. FinBERT를 이용한 기업 감성점수 도출

- XGBoost에서 선택된 기업들에, 제공된 NASDAQ 뉴스의 8월 데이터에서 FinBERT 감성분석을 실시함
- 이때, all_tck_iem_cd 열에 여러 기업이 있는 경우, 뉴스 기사에 여러 기업에 대한 언급이 포함됨
- 그러므로 특정 기업에 대한 감성분석을 실시하려면 해당 기업코드 또는 기업명이 들어간 문단을 추출하는 게 적절하다고 판단함
- FinBERT를 사용한 이유는, 금융 도메인에 특화하여 pre-trained된 모델이기 때문임
- 그 중 ProsusAI의 모델을 사용한 이유는, 해당 모델이 금융 뉴스 문장들로 이루어진 Financial PhraseBank 데이터로 fine-tuning되어, 뉴스 데이터를 분석하기 적합하다고 판단했기 때문

## 함수 생성

In [63]:
import torch
import pandas as pd

In [65]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.2 MB/s eta 0:00:00


In [66]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
finbert_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [67]:
#주식코드를 입력하면 NASDAQ 데이터프레임에서 관련 기사를 찾아주는 함수
def find_rows_by_tck(nasdaq_df, stock_code):
    rows_to_append = []

    for i in range(len(nasdaq_df)):
        lst = nasdaq_df['all_tck_iem_cd'].iloc[i]
        lst = lst.replace('[', '')
        lst = lst.replace(']', '')
        lst = lst.replace("'", '')
        lst = lst.replace(" ", '')
        lst = lst.split(',')

        if stock_code in lst:
            a = nasdaq_df.iloc[i]
            rows_to_append.append(a)

    if rows_to_append:
        nasdaq_rows = pd.concat(rows_to_append, axis=1).T.reset_index(drop=True)
        return nasdaq_rows
    else:
        return pd.DataFrame()

#주어진 데이터프레임의 'text' 열에서 주식코드와 기업명을 포함한 문단을 추출
def get_relevant_paragraphs(nasdaq_row, stock_code, company_name):
    text = nasdaq_row['text']
    paragraphs = text.split('\n\n')
    relevant_paragraphs = [paragraph for paragraph in paragraphs if stock_code in paragraph or company_name in paragraph]
    relevant_paragraphs = '\n\n'.join(relevant_paragraphs)
    return relevant_paragraphs

#추출한 문단에 FinBERT 적용
def finbert(input_text):
    inputs = tokenizer(input_text, padding = True, truncation = True, return_tensors='pt')
    outputs = finbert_model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    positive = predictions[:, 0].tolist()
    negative = predictions[:, 1].tolist()
    neutral = predictions[:, 2].tolist()

    table = {
            "Text": input_text,
            "Positive":positive,
            "Negative":negative,
            "Neutral":neutral}

    score_row = pd.DataFrame(table, columns = ["Text", "Positive", "Negative", "Neutral"])

    return score_row

#위 함수들을 결합하여, 입력한 주식코드에 대한 핀버트 적용
def finbert_from_stock_code(nasdaq_df, stock_code):
    score_df = pd.DataFrame()
    nasdaq_rows = find_rows_by_tck(nasdaq_df, stock_code)

    if nasdaq_rows.empty:
        return score_df

    stock_info_df = pd.read_csv(os.path.join(READ_PATH, "stock_description.csv"))
    company_name = stock_info_df[stock_info_df['tck_iem_cd'] == stock_code]['fc_sec_eng_nm'].iloc[0]

    for index, nasdaq_row in nasdaq_rows.iterrows():
        result = get_relevant_paragraphs(nasdaq_row, stock_code, company_name)
        score_row = finbert(result)

        score_row['rgs_dt'] = nasdaq_row['rgs_dt']
        score_row['all_tck_iem_cd'] = nasdaq_row['all_tck_iem_cd']
        score_row['til_ifo'] = nasdaq_row['til_ifo']

        score_df = pd.concat([score_df, score_row], ignore_index=True)

    cols = ['rgs_dt', 'all_tck_iem_cd', 'til_ifo'] + [col for col in score_df.columns if col not in ['rgs_dt', 'all_tck_iem_cd', 'til_ifo']]
    score_df = score_df[cols]

    score_df = score_df[score_df['Text'].str.len() > 0]

    return score_df


In [69]:
#기업명, 기사 등장 횟수, positive, negative, neutral에 대한 점수 평균 및 각 카테고리가 가장 높았던 기사의 개수로 데이터프레임 생성
columns = ['Company', 'Count', 'Score_Pos_Mean', 'Score_Neg_Mean', 'Score_Neu_Mean', 'Score_Pos_Count', 'Score_Neg_Count', 'Score_Neu_Count']


def calculate_finbert_sentiments(august_nasdaq_df, company_list, PATH=PATH):
    summarized_scores_df = pd.DataFrame(columns=columns) #각 기업별 요약결과를 보여줌
    all_scores_df = pd.DataFrame() #각 기사별 요약결과를 보여줌
    count = 0

    for company in company_list:
        count += 1
        num_articles = len(find_rows_by_tck(august_nasdaq_df, company))
        company_dataframe = pd.DataFrame()
        print(f'기업 {company}의 기사는 {num_articles}개입니다.')

        try:
            score_df = finbert_from_stock_code(august_nasdaq_df, company)
            score_df['Highest_Sentiment'] = score_df[['Positive', 'Negative', 'Neutral']].idxmax(axis=1)

            all_scores_df = pd.concat([all_scores_df, score_df])

            mean_scores = score_df[['Positive', 'Negative', 'Neutral']].mean()
            score_df_encoded = pd.get_dummies(score_df, columns=['Highest_Sentiment'], prefix=['Sentiment']).fillna(0)

            for col in ['Sentiment_Positive', 'Sentiment_Negative', 'Sentiment_Neutral']:
                if col not in score_df_encoded.columns:
                    score_df_encoded[col] = 0

            count_scores = score_df_encoded[['Sentiment_Positive', 'Sentiment_Negative', 'Sentiment_Neutral']].mean()

            print(f"Mean: {mean_scores['Positive']:.4f} {mean_scores['Negative']:.4f} {mean_scores['Neutral']:.4f}")
            print(f"Count: {count_scores['Sentiment_Positive']:.4f} {count_scores['Sentiment_Negative']:.4f} {count_scores['Sentiment_Neutral']:.4f}")

            company_dataframe = pd.DataFrame({
                'Company': [company],
                'Count': [num_articles],
                'Score_Pos_Mean': [mean_scores['Positive']],
                'Score_Neg_Mean': [mean_scores['Negative']],
                'Score_Neu_Mean': [mean_scores['Neutral']],
                'Score_Pos_Count': [count_scores['Sentiment_Positive']],
                'Score_Neg_Count': [count_scores['Sentiment_Negative']],
                'Score_Neu_Count': [count_scores['Sentiment_Neutral']]
            })

            summarized_scores_df = pd.concat([summarized_scores_df, company_dataframe], ignore_index=True)

        except IndexError as e:
            print(f"IndexError occurred for company {company}: {e}")

        #100번마다 ckpt 저장
        # if count % 100 == 0:
        #     print(count)
        #     all_scores_df.to_csv(os.path.join(PATH, f'all_scores_{count}.csv'), index=False)
        #     summarized_scores_df.to_csv(os.path.join(PATH, f'summarized_scores_{count}.csv'), index=False)

    all_scores_df.to_csv(os.path.join(PATH, 'all_scores.csv'), index=False)

    summarized_scores_df.to_csv(os.path.join(PATH, 'summarized_scores.csv'), index=False)

    return all_scores_df, summarized_scores_df


## 핀버트 실행

In [70]:
import torch
import pandas as pd

In [71]:
PATH #경로 확인

'/content/'

In [72]:
nasdaq_df = pd.read_csv(os.path.join(READ_PATH, "nasdaq_final.csv"))

In [73]:
#8월 데이터만 필터링
august_nasdaq_df = nasdaq_df[nasdaq_df['rgs_dt']>=20230801]

In [84]:
all_scores_df, summarized_scores_df = calculate_finbert_sentiments(august_nasdaq_df, financial_filtered_cp_list)

기업 VOD의 기사는 8개입니다.
Mean: 0.5204 0.0188 0.4608
Count: 0.5000 0.0000 0.5000
기업 INTC의 기사는 176개입니다.
Mean: 0.3003 0.2780 0.4217
Count: 0.3019 0.2704 0.4277
기업 NICE의 기사는 6개입니다.
Mean: 0.6676 0.0944 0.2380
Count: 0.8000 0.0000 0.2000
기업 DRS의 기사는 5개입니다.
Mean: 0.5615 0.0401 0.3984
Count: 0.6000 0.0000 0.4000
기업 PERI의 기사는 15개입니다.
Mean: 0.2872 0.1218 0.5910
Count: 0.2143 0.0714 0.7143
기업 LNTH의 기사는 10개입니다.
Mean: 0.1503 0.2433 0.6064
Count: 0.1000 0.2000 0.7000
기업 CSCO의 기사는 111개입니다.
Mean: 0.3515 0.1759 0.4726
Count: 0.3238 0.1619 0.5143
기업 GOOG의 기사는 184개입니다.
Mean: 0.3158 0.1610 0.5232
Count: 0.2899 0.1522 0.5580
기업 SOUN의 기사는 14개입니다.
Mean: 0.4337 0.1466 0.4197
Count: 0.5000 0.1429 0.3571
기업 KTOS의 기사는 13개입니다.
Mean: 0.4447 0.0567 0.4987
Count: 0.4615 0.0000 0.5385
기업 AOSL의 기사는 5개입니다.
Mean: 0.3661 0.3993 0.2346
Count: 0.4000 0.4000 0.2000
기업 NEWT의 기사는 8개입니다.
Mean: 0.5935 0.1067 0.2998
Count: 0.6250 0.1250 0.2500
기업 CCCS의 기사는 6개입니다.
Mean: 0.0869 0.5300 0.3831
Count: 0.0000 0.5000 0.5000
기업 APLD의 기사는 6개입니

In [85]:
filtered_company_df_pos = summarized_scores_df[~(summarized_scores_df==0).any(axis=1)]
filtered_company_df_pos = filtered_company_df_pos[
    (filtered_company_df_pos['Score_Pos_Mean'] - filtered_company_df_pos['Score_Neg_Mean'] > 0) &
    (filtered_company_df_pos['Score_Pos_Count'] - filtered_company_df_pos['Score_Neg_Count'] > 0)
 ]
filtered_company_df_pos = filtered_company_df_pos.sort_values(by='Count', ascending=False)
filtered_company_df_pos

,Company,Count,Score_Pos_Mean,Score_Neg_Mean,Score_Neu_Mean,Score_Pos_Count,Score_Neg_Count,Score_Neu_Count
14,MSFT,434,0.392048,0.147402,0.460549,0.396175,0.128415,0.475410
25,AMD,226,0.369931,0.331374,0.298695,0.398058,0.325243,0.276699
7,GOOG,184,0.315841,0.160963,0.523195,0.289855,0.152174,0.557971
1,INTC,176,0.300308,0.277963,0.421729,0.301887,0.270440,0.427673
6,CSCO,111,0.351525,0.175900,0.472576,0.323810,0.161905,0.514286
4,PERI,15,0.287211,0.121754,0.591035,0.214286,0.071429,0.714286
8,SOUN,14,0.433710,0.146551,0.419739,0.500000,0.142857,0.357143
16,RGTI,10,0.512812,0.308145,0.179043,0.500000,0.400000,0.100000
11,NEWT,8,0.593522,0.106689,0.299788,0.625000,0.125000,0.250000
28,AEHR,7,0.404243,0.148987,0.446769,0.285714,0.142857,0.571429


In [88]:
filtered_company_df_pos.to_csv(os.path.join(PATH, "filtered_company_df_pos.csv"), index=False)

In [86]:
len(filtered_company_df_pos)

10

In [87]:
filtered_company_df_pos['Company'].tolist()

['MSFT', 'AMD', 'GOOG', 'INTC', 'CSCO', 'PERI', 'SOUN', 'RGTI', 'NEWT', 'AEHR']